<a href="https://colab.research.google.com/github/LuigiSigillo/nlp2021-hw/blob/master/nlp2021-hw2/hw2/stud/implementation/nlp_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
from google.colab import drive
# general
import numpy as np
import os
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
from typing import *
import string
import json
import pandas as pd

# torch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader,SequentialSampler
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter

# torch text
!pip install torchtext --upgrade

from torchtext import data
from torchtext.vocab import Vectors

from pprint import pprint
from torchtext.vocab import *
from collections import Counter
import random

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

!pip install pytorch_lightning

import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.optim as optim
from pytorch_lightning.callbacks import ModelCheckpoint

# SKLEARN
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import precision_score as sk_precision

# transformers
!pip install transformers
from transformers import DistilBertTokenizerFast, BertTokenizerFast
from transformers import DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from transformers import DistilBertForTokenClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import BertModel,get_linear_schedule_with_warmup

drive.mount('/content/drive')

Requirement already up-to-date: torchtext in /usr/local/lib/python3.7/dist-packages (0.9.1)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 808kB 7.7MB/s 
     |████████████████████████████████| 829kB 48.2MB/s 
     |████████████████████████████████| 645kB 45.6MB/s 
     |████████████████████████████████| 10.6MB 48.8MB/s 
     |████████████████████████████████| 276kB 53.8MB/s 
     |████████████████████████████████| 112kB 56.9MB/s 
     |████████████████████████████████| 1.3MB 34.7MB/s 
     |████████████████████████████████| 296kB 47.7MB/s 
     |████████████████████████████████| 143kB 58.2MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070

     |████████████████████████████████| 2.3MB 8.7MB/s 
     |████████████████████████████████| 3.3MB 50.4MB/s 
     |████████████████████████████████| 901kB 52.1MB/s 
Mounted at /content/drive


Set up of seeds and  folders


In [2]:
root_folder = '/content/drive/My Drive/NLP/nlp2021-hw2'
dataset_folder = os.path.join(root_folder,'data')

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True  

model_b = False

!nvidia-smi

Mon Jun  7 11:01:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# TASK A-B (Bi-LSTM)

## Dataset

In [83]:
class BiLSTMDataset(Dataset):

    def __init__(self, 
                 data_path:str,
                 window_size:int, 
                 window_shift:int=-1,
                 model_b:bool=False,
                 device="cuda"):
        """
        Args:
            data_path (string): The path containing the data
            window_size (integer): The maximum length of a sentence in terms of number of tokens.
            window_shift (integer): The number of tokens we shift the window over the sentence. Default value is -1 meaning that the window will be shifted by window_size.
            model_b:bool=False,

        """

        self.window_size = window_size
        self.window_shift = window_shift if window_shift > 0 else window_size
        self.model_b = model_b

        sentences = self.load_data(data_path)
        self.device = device
        self.data = self.create_windows(sentences)
        self.encoded_data = None
    

    def remove_stopwords(self,sent: str) -> str:
        stop_words = set(stopwords.words('english'))

        # remove punkt
        others = "–" +"—" + "−" + "’" + "”" + "“" #These chars arent inside the standard punctuation
        str_punkt = string.punctuation+ others
        translator = str.maketrans(str_punkt, ' '*len(str_punkt)) 
        word_tokens = word_tokenize(sent.translate(translator)) 
        
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        return filtered_sentence

    def load_data(self,data_path):
        """ 
        Args:
            data_path 
        """
        sentences = []
        with open(data_path) as json_file:
            list_of_sentences = json.load(json_file)
            for obj in list_of_sentences:
                _sentence = []
                #lemmatized = [lemmatizer.lemmatize(w)  for w in obj['text'].split(" ")]
                lemmatized = self.remove_stopwords(obj['text'])
                for t in lemmatized:
                    ne_label = "O"
                    sentiment = ""
                    for i in range(len(obj['targets'])):
                        lemmatized_target = lemmatizer.lemmatize(obj['targets'][i][1])
                        if t in lemmatized_target:
                            ne_label = "B" if t == lemmatized_target.split(" ")[0] else "I"
                            try:
                                if ne_label == "I" and _sentence[-1]["ne_label"] == "O":
                                    ne_label = "O"
                            except:
                                ne_label = "O"
                                
                            sentiment = obj['targets'][i][2]
                            #embed sentiment directly into the position of word
                            if self.model_b and ne_label != "O":
                                ne_label = ne_label +"-"+sentiment
                    token = {"token": t, "ne_label": ne_label , "sentiment" :sentiment}
                    _sentence.append(token)
                sentences.append(_sentence)
            return sentences

    def create_windows(self, sentences): # to save space in memory ? why we do not load directly all the sentence?
        """ 
        Args:
            sentences (list of lists of dictionaries, where each dictionary represents a word occurrence)
        """
        data = []
        for sentence in sentences:
            for i in range(0, len(sentence), self.window_shift):
                window = sentence[i:i+self.window_size]
                if len(window) < self.window_size:
                    window = window + [None]*(self.window_size - len(window))  # to match the same length of sentences
                assert len(window) == self.window_size
                data.append(window)
        return data


    def index_dataset(self, l_vocabulary, l_label_vocabulary):
        self.encoded_data = list()
        for i in range(len(self.data)):
            # for each window
            sentence = self.data[i]
            encoded_sentence = torch.LongTensor(self.encode_text(sentence, l_vocabulary)).to(self.device)
            
            # for each element d in the elem window (d is a dictionary with the various fields from the CoNLL line) 
            encoded_labels = torch.LongTensor([l_label_vocabulary[d["ne_label"]] if d is not None else l_label_vocabulary["<pad>"] for d in sentence]).to(self.device)
            
            self.encoded_data.append({"inputs":encoded_sentence, "outputs":encoded_labels})


    @staticmethod
    def encode_text(sentence:list, l_vocabulary:Vocab):
        """
        Args:
            sentences (list): list of OrderedDict, each carrying the information about one token.
            l_vocabulary (Vocab): vocabulary with mappings from words to indices and viceversa.
        Return:
            The method returns a list of indices corresponding to the input tokens.
        """
        indices = list()
        for w in sentence:
            if w is None:
                indices.append(l_vocabulary["<pad>"])
            elif w["token"] in l_vocabulary.stoi: # vocabulary string to integer (necessary to search faster)
                indices.append(l_vocabulary[w["token"]])
            else:
                indices.append(l_vocabulary["<unk>"])
        return indices

    
    @staticmethod
    def decode_output(outputs:torch.Tensor,
                    l_label_vocabulary: Vocab):
        """
        Args:
            outputs (Tensor): a Tensor with shape (batch_size, max_len, label_vocab_size)
                containing the logits outputed by the neural network.
            l_label_vocabulary (Vocab): is the vocabulary containing the mapping from
            a string label to its corresponding index and vice versa
        Output:
            The method returns a list of batch_size length where each element is a list
            of labels, one for each input token.
        """
        max_indices = torch.argmax(outputs, -1).tolist() # shape = (batch_size, max_len)
        predictions = list()
        for indices in max_indices:
            # vocabulary integer to string is used to obtain the corresponding word from the max index
            predictions.append([l_label_vocabulary.itos[i] for i in indices])
        return predictions

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.encoded_data[idx]
    
    def get_raw_element(self, idx):
        return self.data[idx]

testing dataset

In [84]:
training_file = dataset_folder+"/laptops_train.json"

def test_dataset_class(training_file):
    
    window_size, window_shift = 10, 10

    dataset = BiLSTMDataset(training_file, window_size, window_shift, model_b)
    
    print('Dataset test:')
    for i in range(10):
        print('  sample {}: {}'.format(i, [t["token"] + ":" +  t["ne_label"] + "-"+ t["sentiment"] for t in dataset.get_raw_element(i) if t is not None]))

test_dataset_class(training_file)

Dataset test:
  sample 0: ['always:O-', 'use:O-', 'backup:O-', 'hard:B-neutral', 'disk:I-neutral', 'store:O-', 'important:O-', 'files:O-', 'times:O-']
  sample 1: ['also:O-', 'love:O-', 'small:O-', 'convenient:O-', 'size:B-positive', 'laptop:O-', 'making:O-', 'perfect:O-', 'tool:O-', 'academic:O-']
  sample 2: ['studies:O-']
  sample 3: ['thought:O-', 'white:O-', 'Mac:O-', 'computers:O-', 'looked:O-', 'dirty:O-', 'quicly:O-', 'use:O-neutral', 'mousepad:B-neutral', 'place:O-']
  sample 4: ['hands:O-', 'typing:O-']
  sample 5: ['always:O-', 'reliable:O-', 'never:O-', 'bugged:O-', 'responds:B-positive', 'well:O-']
  sample 6: ['real:O-', 'stand:O-', 'computer:O-', 'feel:O-', 'keyboard:B-positive', 'speed:B-positive']
  sample 7: ['Called:O-', 'headquarters:O-', 'report:O-', 'TFT:B-negative', 'panel:I-negative', 'broken:O-', 'fixed:O-', 'end:O-', 'week:O-', 'week:O-']
  sample 8: ['3:O-']
  sample 9: ['computer:O-', 'bought:O-', 'wanted:O-', 'top:O-', 'line:O-', 'fast:O-', 'reliable:O-', '

Data module definition

In [85]:
def create_windows(sentences): # to save space in memory ? why we do not load directly all the sentence?
        """ 
        Args:
            sentences (list of lists of dictionaries, where each dictionary represents a word occurrence)
        """
        data = []
        window_size = 100
        for sentence in sentences:
            for i in range(0, len(sentence), window_shift):
                window = sentence[i:i+window_size]
                if len(window) < window_size:
                    window = window + [None]*(window_size - len(window))  # to match the same length of sentences
                assert len(window) == window_size
                data.append(window)
        return data

class DataModuleBiLSTM(pl.LightningDataModule):
    def __init__(self, training_file, dev_file, window_size, window_shift, vocabulary, label_vocabulary, model_b=None):
        super().__init__()
        self.training_file = training_file
        self.dev_file = dev_file
        self.window_size = window_size
        self.window_shift = window_shift
        self.vocabulary = vocabulary
        self.label_vocabulary = label_vocabulary
        self.model_b = model_b

    def setup(self, stage=None):
      self.trainingset = BiLSTMDataset(self.training_file, self.window_size, self.window_shift, self.model_b)
      self.devset = BiLSTMDataset(self.dev_file, self.window_size, self.window_shift, self.model_b)

      self.trainingset.index_dataset(self.vocabulary, self.label_vocabulary)
      self.devset.index_dataset(self.vocabulary, self.label_vocabulary)
          
    def train_dataloader(self):
      return DataLoader(self.trainingset, batch_size=128)
    
    def val_dataloader(self):
        return DataLoader(self.devset, batch_size=128)

### Building vocabulary

In [86]:
def build_vocab(dataset, min_freq=1):
    counter = Counter()
    for i in tqdm(range(len(dataset))):
        # for each token in the sentence viewed as a dictionary of items from the line
        for token in dataset.get_raw_element(i):
            if token is not None:
                counter[token["token"]]+=1
    #Counter({'the': 1046, 'and': 671, 'to': 604, etc....}
    # we add special tokens for handling padding and unknown words at testing time.

    return Vocab(counter, specials=['<pad>', '<unk>'], min_freq=min_freq)

def build_label_vocab(dataset):
    counter = Counter()
    for i in tqdm(range(len(dataset))):
        for token in dataset.get_raw_element(i):
            if token is not None:
                counter[token["ne_label"]]+=1
    # No <unk> token for labels. Counter({'O': 19179, 'B': 1548, 'I': 718})
    print(counter)
    return Vocab(counter, specials=['<pad>'])





In [87]:
training_file = dataset_folder+"/laptops_train.json"
dev_file = dataset_folder+"/laptops_dev.json"
window_size, window_shift = 100, 100
dataset = BiLSTMDataset(training_file, window_size, window_shift, model_b)
vocabulary = build_vocab(dataset)
label_vocabulary = build_label_vocab(dataset)
dataset.index_dataset(vocabulary, label_vocabulary)



100%|██████████| 2497/2497 [00:00<00:00, 113784.46it/s]


100%|██████████| 2497/2497 [00:00<00:00, 162465.52it/s]


Counter({'O': 15738, 'B': 1705, 'I': 820})


## Usage of pretrained embeddings

In [ ]:
vocabulary.load_vectors("glove.6B.50d")
#vocabulary.load_vectors("charngram.100d")
#vocabulary.load_vectors("fasttext.en.300d")



In [ ]:
print(vocabulary.vectors[1165])
#print(vocabulary.vectors[0].get_vecs_by_tokens("pain"))

## Model

In [88]:
class ABSAModel(pl.LightningModule):
    def __init__(self, hparams, embeddings = None, comments="without_pretrained_embeddings",*args, **kwargs):
        super(ABSAModel, self).__init__(*args, **kwargs)

        self.save_hyperparameters(hparams)
        self.loss_function = nn.CrossEntropyLoss(ignore_index=label_vocabulary['<pad>'])
        #self.model = ABSATaggerModel(self.hparams, embeddings)
        self.word_embedding = nn.Embedding(self.hparams.vocab_size, self.hparams.embedding_dim) # hparams.vocab_size words in vocab, hparams.embedding_dim dimensional embeddings

        if embeddings is not None:
            print("initializing embeddings from pretrained")
            self.word_embedding = nn.Embedding.from_pretrained(embeddings)

        self.lstm = nn.LSTM(self.hparams.embedding_dim, self.hparams.hidden_dim, 
                            bidirectional=self.hparams.bidirectional,
                            num_layers=self.hparams.num_layers, 
                            dropout = self.hparams.dropout if self.hparams.num_layers > 1 else 0)
        
        lstm_output_dim = self.hparams.hidden_dim if self.hparams.bidirectional is False else self.hparams.hidden_dim * 2

        # During training, randomly zeroes some of the elements of the 
        # input tensor with probability hparams.dropout. 
        # This has proven to be an effective technique for regularization and 
        # preventing the co-adaptation of neurons
        self.dropout = nn.Dropout(self.hparams.dropout)
        self.classifier = nn.Linear(lstm_output_dim, self.hparams.num_classes)

        self.writer = SummaryWriter(comment=comments+"_modelb="+str(model_b))
        self.epoch_t, self.epoch_ev = -1,-1

    def forward(self, x):
        embeddings = self.word_embedding(x)
        embeddings = self.dropout(embeddings)
        o, (h, c) = self.lstm(embeddings)
        o = self.dropout(o)
        logits = self.classifier(o)
        
        predictions = torch.argmax(logits, -1)
        
        return logits, predictions

    def training_step(self, batch, batch_nb):
        '''
        {
        'inputs': tensor([  5, 121,  34,   6, 834,  68, 307,   4, 370, 684, 663,  40,  42, 748, 0,   0,   0,   0,   0], device='cuda:0'), 
        'outputs': tensor([1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
        }
        '''
        inputs = batch['inputs']
        labels = batch['outputs']
        # We receive one batch of data and perform a forward pass:
        logits, _ = self.forward(inputs)

        # We adapt the logits and labels to fit the format required for the loss function
        logits = logits.view(-1, logits.shape[-1])
        labels = labels.view(-1)

        loss = self.loss_function(logits, labels)
        # Log it:
        self.log('train_loss', loss, prog_bar=True)
        if self.epoch_t != self.current_epoch:
            self.epoch_t = self.current_epoch
            self.writer.add_scalar("train/loss", loss, self.current_epoch)
            self.log_f1(logits,labels,"train")


        # loss that will be used to update the weights:
        return loss

    def validation_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']

        logits, _ = self.forward(inputs)
        logits = logits.view(-1, logits.shape[-1])
        labels = labels.view(-1)
        sample_loss = self.loss_function(logits, labels)
        self.log('valid_loss', sample_loss, prog_bar=True)
        
        if self.epoch_ev != self.current_epoch:
            self.epoch_ev = self.current_epoch
            self.writer.add_scalar("eval/loss", sample_loss, self.current_epoch)
            self.log_f1(logits,labels,"eval")


    def configure_optimizers(self):
        return optim.Adam(self.parameters())


    def log_f1(self,logits,indexed_labels,mode="train"):
        predictions = torch.argmax(logits, -1).view(-1)
        labels = indexed_labels.view(-1)
        valid_indices = labels != 0
        
        valid_predictions = predictions[valid_indices]
        valid_labels = labels[valid_indices]
        macro_f1 = f1_score(valid_labels.tolist(), valid_predictions.tolist(), average="macro", zero_division=0)
        self.writer.add_scalar(mode+"/f1", macro_f1, self.current_epoch)


## Training

In [99]:
to_be_saved = False
try:
    embedding_dim = vocabulary.vectors.shape[1]
except:
    embedding_dim = 100
hparams = {'vocab_size': len(vocabulary),
            'hidden_dim': 80,
            'embedding_dim': embedding_dim,
            'num_classes': len(label_vocabulary), # number of different universal POS tags
            'bidirectional': True,
            'num_layers': 2,
            'dropout': 0.2}
window_size, window_shift = 100, 100
data_module = DataModuleBiLSTM(training_file, dev_file, window_size, window_shift, vocabulary, label_vocabulary, model_b)
trainer = pl.Trainer(gpus=1, val_check_interval=1.0, max_epochs=15)

model = ABSAModel(hparams,comments="without_pretrained")
#model = ABSAModel(hparams,vocabulary.vectors,comments="with_charngram")
trainer.fit(model, datamodule=data_module)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_function  | CrossEntropyLoss | 0     
1 | word_embedding | Embedding        | 414 K 
2 | lstm           | LSTM             | 271 K 
3 | dropout        | Dropout          | 0     
4 | classifier     | Linear           | 644   
----------------------------------------------------
686 K     Trainable params
0         Non-trainable params
686 K     Total params
2.745     Total estimated model params size (MB)


## Evaluation

In [ ]:
try:
    %reload_ext tensorboard
except:
    %load_ext tensorboard
%tensorboard --logdir=runs/

In [90]:
from sklearn.metrics import precision_score as sk_precision
from sklearn.metrics import f1_score

def compute_scores(model:pl.LightningModule, l_dataset:DataLoader, l_label_vocab:Vocab):
    model.freeze()
    model.cuda()
    all_predictions = list()
    all_labels = list()
    for indexed_elem in l_dataset:
        indexed_in = indexed_elem["inputs"]
        indexed_labels = indexed_elem["outputs"]
        predictions, _ = model(indexed_in)
        predictions = torch.argmax(predictions, -1).view(-1)
        labels = indexed_labels.view(-1)
        valid_indices = labels != 0
        
        valid_predictions = predictions[valid_indices]
        valid_labels = labels[valid_indices]
        
        all_predictions.extend(valid_predictions.tolist())
        all_labels.extend(valid_labels.tolist())

    macro_accuracy = accuracy_score(all_labels, all_predictions)
    macro_f1 = f1_score(all_labels, all_predictions, average="macro", zero_division=0)
    per_class_precision = sk_precision(all_labels, all_predictions, labels = list(range(len(l_label_vocab))), average=None, zero_division=0)
    model.unfreeze()
    return {"macro_accuracy":macro_accuracy,
            "f1_macro":macro_f1, 
            "per_class_precision":per_class_precision}

In [100]:
scores = compute_scores(model, data_module.val_dataloader(), label_vocabulary)
per_class_precision = scores["per_class_precision"]
print("Accuracy: {}\nMacro F1: {}".format(scores["macro_accuracy"], scores["f1_macro"]))
print("Per class Precision:")
for idx_class, precision in sorted(enumerate(per_class_precision), key=lambda elem: -elem[1]):
    label = label_vocabulary.itos[idx_class]
    print(label, precision)

if to_be_saved:
    torch.save(model.state_dict(), root_folder+'/model/model_b={}_f1_{:0.4f}.pt'.format(str(model_b), scores["f1_macro"])) # save the model state


Accuracy: 0.9076615922136262
Macro F1: 0.6419478355669707
Per class Precision:
O 0.9311385459533608
I 0.6756756756756757
B 0.6701388888888888
<pad> 0.0


## docker

In [102]:
class PreprocessAB():
    def __init__(self,sentences_list):
        self.window_size = 100
        self.window_shift = 100
        self.sentences = self.load_data(sentences_list)
        self.data = self.create_windows(self.sentences)
        self.encoded_data = None
        self.vocabulary = self.build_vocab()
        self.label_vocabulary = self.build_label_vocab()
        self.index_dataset(self.vocabulary, self.label_vocabulary)

    def remove_stopwords(self,sent: str) -> str:
        stop_words = set(stopwords.words('english'))

        # remove punkt
        others = "–" +"—" + "−" + "’" + "”" + "“" #These chars arent inside the standard punctuation
        str_punkt = string.punctuation+ others
        translator = str.maketrans(str_punkt, ' '*len(str_punkt)) 
        word_tokens = word_tokenize(sent.translate(translator)) 
        
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        return filtered_sentence

    def load_data(self,list_of_sentences):
        sentences = []
        for obj in list_of_sentences:
            _sentence = []
            # lemmatized = [lemmatizer.lemmatize(w)  for w in obj['text'].split(" ")]
            #lemmatized = self.remove_stopwords(" ".join(lemmatized))
            lemmatized = self.remove_stopwords(obj['text'])

            for t in lemmatized:
                token = {"token": t}
                _sentence.append(token)
            sentences.append(_sentence)
        return sentences

    def create_windows(self, sentences): # to save space in memory ? why we do not load directly all the sentence?
        """ 
        Args:
            sentences (list of lists of dictionaries, where each dictionary represents a word occurrence)
        """
        data = []
        for sentence in sentences:
            for i in range(0, len(sentence), self.window_shift):
                window = sentence[i:i+self.window_size]
                if len(window) < self.window_size:
                    window = window + [None]*(self.window_size - len(window))  # to match the same length of sentences
                assert len(window) == self.window_size
                data.append(window)
        return data


    def index_dataset(self, l_vocabulary, l_label_vocabulary):
        self.encoded_data = list()
        for i in range(len(self.data)):
            # for each window
            sentence = self.data[i]
            encoded_sentence = torch.LongTensor(self.encode_text(sentence, l_vocabulary)).cpu()
            
           
            self.encoded_data.append({"inputs":encoded_sentence})


    @staticmethod
    def encode_text(sentence:list, l_vocabulary:Vocab):
        """
        Args:
            sentences (list): list of OrderedDict, each carrying the information about one token.
            l_vocabulary (Vocab): vocabulary with mappings from words to indices and viceversa.
        Return:
            The method returns a list of indices corresponding to the input tokens.
        """
        indices = list()
        for w in sentence:
            if w is None:
                indices.append(l_vocabulary["<pad>"])
            elif w["token"] in l_vocabulary.stoi: # vocabulary string to integer (necessary to search faster)
                indices.append(l_vocabulary[w["token"]])
            else:
                indices.append(l_vocabulary["<unk>"])
        return indices

    
    @staticmethod
    def decode_output(outputs:torch.Tensor,
                    l_label_vocabulary: Vocab):
        """
        Args:
            outputs (Tensor): a Tensor with shape (batch_size, max_len, label_vocab_size)
                containing the logits outputed by the neural network.
            l_label_vocabulary (Vocab): is the vocabulary containing the mapping from
            a string label to its corresponding index and vice versa
        Output:
            The method returns a list of batch_size length where each element is a list
            of labels, one for each input token.
        """
        max_indices = torch.argmax(outputs, -1).tolist() # shape = (batch_size, max_len)
        predictions = list()
        for indices in max_indices:
            # vocabulary integer to string is used to obtain the corresponding word from the max index
            predictions.append([l_label_vocabulary.itos[i] for i in indices])
        return predictions

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.encoded_data[idx]
    
    def get_raw_element(self, idx):
        return self.data[idx]

    def build_vocab(self, min_freq=1):
        counter = Counter()
        for i in tqdm(range(len(self.data))):
            # for each token in the sentence viewed as a dictionary of items from the line
            for token in self.get_raw_element(i):
                if token is not None:
                    counter[token["token"]]+=1
        #Counter({'the': 1046, 'and': 671, 'to': 604, etc....}
        # we add special tokens for handling padding and unknown words at testing time.

        return Vocab(counter, specials=['<pad>', '<unk>'], min_freq=min_freq)

    def build_label_vocab(self):
        counter = Counter({'O': 16101, 'B-positive': 758, 'B-negative': 694, 'B-neutral': 346, 'I-negative': 317, 'I-positive': 242, 'I-neutral': 226, 'B-conflict': 33, 'I-conflict': 15})
        return Vocab(counter, specials=['<pad>'])


def predict_together_AB(samples: List[Dict]) -> List[Dict]:
    targets = []
    prep = PreprocessAB(samples)
    for i,test_elem in tqdm(enumerate(prep.encoded_data)):
        json_pred = {"targets":[]}
        lst,pred = [],[]
        test_x = test_elem["inputs"].to("cpu")
        logits, predictions = model(test_x.unsqueeze(0))
        decoded_labels = prep.decode_output(logits, label_vocabulary)[0]
        idxs = test_x.tolist()
        for j,word in enumerate(decoded_labels):
            if word.startswith("B"):
                json_pred["targets"].append((prep.vocabulary.itos[idxs[j]],word.split("-")[1]))
            elif (word.startswith("I")) and (decoded_labels[j-1].startswith("B")):
                try:
                    last_tuple = json_pred['targets'][-1]
                    words_tagged = last_tuple[0] + " " + prep.vocabulary.itos[idxs[j]]
                    sent_tagged = last_tuple[1]
                    json_pred['targets'][-1] = (words_tagged, sent_tagged)
                except:
                    words_tagged = prep.vocabulary.itos[idxs[j]]
                    sent_tagged = word.split("-")[1]
                    json_pred['targets'].append((words_tagged, sent_tagged))
            elif word.startswith("I"):
                json_pred["targets"].append((prep.vocabulary.itos[idxs[j]],word.split("-")[1]))

        targets.append(json_pred)
    #print(targets)
    return targets



def predict_a_then_b(samples):
    targets = []
    prep = PreprocessAB(samples)
    for i,test_elem in tqdm(enumerate(prep.encoded_data)):
        json_pred = {"targets":[]}
        lst,pred = [],[]
        test_x = test_elem["inputs"].to("cpu")
        logits, predictions = model(test_x.unsqueeze(0))
        decoded_labels = prep.decode_output(logits, label_vocabulary)[0]
        idxs = test_x.tolist()
        # for a,b in zip(decoded_labels,idxs):
        #     print(a,"\t",prep.vocabulary.itos[b])
        for j,word in enumerate(decoded_labels):
            if word == "B":
                json_pred["targets"].append((prep.vocabulary.itos[idxs[j]],"positive"))
            elif (word == "I") and (decoded_labels[j-1] == "B"):
                try:
                    last_tuple = json_pred['targets'][-1]
                    words_tagged = last_tuple[0] + " " + prep.vocabulary.itos[idxs[j]]
                    sent_tagged = last_tuple[1]
                    json_pred['targets'][-1] = (words_tagged, sent_tagged)
                except:
                    words_tagged = prep.vocabulary.itos[idxs[j]]
                    sent_tagged = "positive"
                    json_pred['targets'].append((words_tagged, sent_tagged))
            elif word == "I":
                json_pred["targets"].append((prep.vocabulary.itos[idxs[j]],"positive"))

        targets.append(json_pred)

    return targets
def load_data(data_path):
    with open(data_path) as json_file:
        list_of_sentences = json.load(json_file)
    return list_of_sentences


a = load_data(dataset_folder+"/laptops_dev.json")
#a = a + load_data(dataset_folder+"/restaurants_dev.json")
random.shuffle(a)
model.cpu()
#t = predictB(a[:30])
t = predict_a_then_b(a)
for gt,pred in zip(a[:40],t[:40]):
    print(gt['targets'],"SEP",pred['targets'])
evaluate_extraction(a,t)
evaluate_sentiment(a,t)



100%|██████████| 545/545 [00:00<00:00, 94470.21it/s]


0it [00:00, ?it/s]

49it [00:00, 484.32it/s]

98it [00:00, 484.62it/s]

146it [00:00, 481.33it/s]

195it [00:00, 482.39it/s]

245it [00:00, 484.61it/s]

293it [00:00, 481.18it/s]

341it [00:00, 478.81it/s]

390it [00:00, 481.85it/s]

442it [00:00, 492.14it/s]

493it [00:01, 497.20it/s]

545it [00:01, 489.12it/s]

[] SEP [('s5988', 'positive')]
[[[142, 150], 'programs', 'negative'], [[155, 162], 'drivers', 'negative']] SEP [('first', 'positive'), ('issues', 'positive'), ('powering', 'positive'), ('issues', 'positive')]
[[[31, 39], 'features', 'positive']] SEP []
[[[52, 72], 'sound output quality', 'negative']] SEP []
[[[33, 37], 'BIOS', 'negative']] SEP []
[] SEP [('ago', 'positive')]
[] SEP []
[] SEP [('desktop', 'positive')]
[] SEP []
[] SEP [('took', 'positive'), ('bag', 'positive')]
[[[39, 43], 'work', 'negative']] SEP []
[[[30, 34], 'cost', 'positive']] SEP []
[] SEP [('MAC', 'positive')]
[] SEP [('fan', 'positive')]
[] SEP [('times', 'positive')]
[] SEP []
[] SEP [('go', 'positive')]
[] SEP [('husband', 'positive')]
[[[110, 117], 'company', 'negative']] SEP [('another', 'positive')]
[] SEP [('love', 'positive')]
[[[41, 49], 'programs', 'negative']] SEP [('numbers', 'positive'), ('numbers', 'positive'), ('things', 'positive')]
[] SEP []
[[[22, 33], 'gaming look', 'positive'], [[70, 88], 'ga

# TASK B (Bi-LSTM (WiC))

##Dataset

In [ ]:
class TaskBDataset(torch.utils.data.Dataset):

    def __init__(self, dataset_path: str):
        self.data_store,self.sentences = self.load_data(dataset_path)
        self.vocabulary = self.build_vocab(self.sentences)
        self.convert_all_2_indices()
        self.mlb = MultiLabelBinarizer()
        self.encode_tags()


    def encode_tags(self):
        self.tags = self.mlb.fit_transform([c for (a,b,c) in self.data_store])
        self.data_store = [(a,b, torch.tensor(self.tags[i]) ) for i,(a,b,c) in enumerate(self.data_store)]
    
    def remove_stopwords(self,sent: str) -> str:
        stop_words = set(stopwords.words('english'))

        # remove punkt
        others = "–" +"—" + "−" + "’" + "”" + "“" #These chars arent inside the standard punctuation
        str_punkt = string.punctuation+ others
        translator = str.maketrans(str_punkt, ' '*len(str_punkt)) 
        word_tokens = word_tokenize(sent.translate(translator)) 
        
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        return filtered_sentence

    def load_data(self,data_path):
        data_store,sentences = [],[]
        with open(data_path) as json_file:
            list_of_sentences = json.load(json_file)
            for obj in list_of_sentences:
                _sentence = []
                obj['targets'] = sorted(obj['targets'], key=lambda x: x[0][0])
                sentiments = [obj['targets'][j][2] for j in range(len(obj['targets']))]
 
                for i,targ_obj in enumerate(obj['targets']):
                    #print(targ_obj)
                    new_sent = obj['text'][:targ_obj[0][0]-1]+" <START> " + obj['text'][targ_obj[0][0]:targ_obj[0][1]] + " <END>" + obj['text'][targ_obj[0][1]:]
                    new_sent = [lemmatizer.lemmatize(w)  for w in new_sent.split(" ")]
                    new_sent = " ".join(self.remove_stopwords(" ".join(new_sent)))    
                    index = self.find_indices(new_sent)
                        
                    sentences.append(new_sent)
                    sentiments_converted = [sentiments[i]]
                    data_store.append((new_sent,torch.tensor(index,dtype=torch.long), sentiments_converted))

                if len(obj['targets'])==0:
                    sentiments_converted= ["NONE"]
                    new_sent = obj['text']
                    new_sent = [lemmatizer.lemmatize(w)  for w in new_sent.split(" ")]
                    new_sent = " ".join(self.remove_stopwords(" ".join(new_sent)))
                    index = [0,0]
                    sentences.append(new_sent)
                    data_store.append((new_sent,torch.tensor(index,dtype=torch.long), sentiments_converted))
            return data_store,sentences


    def convert_all_2_indices(self):
        self.data_store = [(self.sentence2indices(a),b,c) for (a,b,c) in self.data_store]
    
    def find_indices(self,new_sent):
        splitted = new_sent.split(" ")
        indices = [i+1 for i,w in enumerate(splitted) if (w=="START") or (w=="END")]
        indices[1] = indices[1]-1
        return indices
    
    def build_vocab(self, dataset, min_freq=1):
        counter = Counter()
        for a in dataset:
            for token in a.split(" "):
                if token is not None:
                    counter[token]+=1
        #Counter({'the': 1046, 'and': 671, 'to': 604, etc....}
        vocabulary = Vocab(counter, specials=['<pad>', '<unk>'], min_freq=min_freq)
        vocabulary.load_vectors("glove.6B.50d")
        return vocabulary

    def sentence2indices(self,sentence: str) -> torch.Tensor:
        return torch.tensor([self.vocabulary[word] for word in sentence.split(' ') if word != ''], dtype=torch.long)

    def decode_tag(self,tag):
        return self.mlb.inverse_transform(tag.reshape(1,-1))
    
    def __len__(self) -> int:
        return len(self.sentences)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.data_store[idx]

In [ ]:
def rnn_collate_fn(data_elements: List[Tuple[torch.Tensor, list]] # list of (x, y,z) pairs
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    X = [de[0] for de in data_elements]  # list of index tensors
    X = torch.nn.utils.rnn.pad_sequence(X, batch_first=True, padding_value=0)  #  shape (batch_size x max_seq_len)
    
    keyword_position = [de[1] for de in data_elements] # list of tuples indices where keyword is [[1st keyword],[2nd keyword]]

    keyword_position = torch.nn.utils.rnn.pad_sequence(keyword_position, batch_first=True, padding_value=0) 

    y = [de[2] for de in data_elements]
    y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=-1) 

    return X, keyword_position, y

In [ ]:
class TaskBDataModule(nn.Module):

    def __init__(
        self, 
        data_train_path: str,
        data_dev_path: str,
        batch_size: int,
        collate_fn=None
    ) -> None:
        super().__init__()
        self.data_train_path = data_train_path
        self.data_dev_path = data_dev_path
        self.batch_size = batch_size
        self.collate_fn = collate_fn

        self.train_dataset = None
        self.validation_dataset = None

    def train_dataloader(self, *args, **kwargs) -> DataLoader:
        self.train_dataset = TaskBDataset(self.data_train_path)
        return DataLoader(self.train_dataset, batch_size=self.batch_size, collate_fn=self.collate_fn)

    def val_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        self.validation_dataset = TaskBDataset(self.data_dev_path)
        return DataLoader(self.validation_dataset, batch_size=self.batch_size,collate_fn=self.collate_fn)


## Training 

Recurrent classifier definition with a customized forward pass

In [ ]:
class TaskBRecurrentClassifier(nn.Module):

    def __init__(
        self,
        vectors_store: torch.Tensor,
        n_hidden: int,
        drop_prob: float,
        bidir: bool,
        n_layer_lstm: int,
        vocab_size:int, 
        embedding_dim: int =100,
    ) -> None:
        super().__init__()


        # embedding layer
        
        self.embedding = torch.nn.Embedding.from_pretrained(vectors_store)
        #self.embedding = nn.Embedding(vocab_size, embedding_dim) # hparams.vocab_size words in vocab, hparams.embedding_dim dimensional embeddings

        self.n_hidden = n_hidden
        # recurrent layer
        self.lstm = torch.nn.LSTM(input_size=vectors_store.shape[1],
                                  hidden_size=n_hidden,
                                  num_layers=n_layer_lstm, 
                                  batch_first=True,
                                  bidirectional=bidir)

        # classification 
        if bidir:
           n_hidden = n_hidden*2
        self.lin1 = torch.nn.Linear(n_hidden, n_hidden)
        self.classifier = torch.nn.Linear(n_hidden, 5)

        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'


    def forward(
        self, 
        X: torch.Tensor, 
        indices_keyword: torch.Tensor, 
        y: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:

        embedding_out = self.embedding(X)
        # recurrent encoding
        lstm_output = self.lstm(embedding_out)[0]
        
        batch_size, seq_len, hidden_size = lstm_output.shape

        #sequence of batch x seq_len vectors 
        flat_output = lstm_output.reshape(-1, hidden_size)
        
        # start offsets of each element in the batch
        sequences_offsets = torch.arange(batch_size, device=self.device) * seq_len
        
        summary_vectors_indices_sent1 = self.get_indices_keyword(indices_keyword, sequences_offsets,0)
        summary_vectors_indices_sent2 = self.get_indices_keyword(indices_keyword, sequences_offsets,1)
        
        # we retrieve the vector of the corrseponding states for the keyword given for each sentence.
          
        summary_vectors_sent1 = flat_output[summary_vectors_indices_sent1]
        summary_vectors_sent2 = flat_output[summary_vectors_indices_sent2]
        
        # do the multiplication of these two vectors retrieved
        summary_vectors = summary_vectors_sent1 * summary_vectors_sent2
        
        # feedforward MLP
        out = self.lin1(summary_vectors)
        out = F.leaky_relu(out)
        
        logits = self.classifier(out)
        
        pred = torch.argmax(logits, -1)
        
        result = {'logits': logits, 'pred': pred} 

        # compute loss
        if y is not None:
            #logits = logits.view(-1, logits.shape[-1])
            #y = y.view(-1)
            try:
                loss = self.loss(logits, torch.tensor(y, dtype=torch.float))
            except:
                print(logits.shape,y.shape)
            result['loss'] = loss
        return result
        
       
    def loss(self, pred, y):
        return self.loss_fn(pred, y)
   
    # '''
    # return the corresponding position of the indices of the keyword
    # summary  = [   0,   57,  114,  171,  228, ...] 
    # indices_keywords = [ [ 21],[ 4],[ 61],[ 22], ...]
    # '''
    # def get_indices_keyword(self,indices_keywords, summary: Sequence[int] ,flat_output) -> torch.Tensor:
    #     res = flat_output[torch.tensor([item for item in indices_keywords]).to(self.device) + summary]
    #     try:
    #         return res
    #     except:
    #         return flat_output[summary]

    '''
    return the corresponding position of the indices of the keywords, for the sent_num passed, so the first if 0 is passed and the second if 2 is passed
    summary  = [   0,   57,  114,  171,  228, ...] 
    indices_keywords = [ [ 6, 21],[ 4, 22],[ 6, 21],[ 4, 22], ...]
    '''
    def get_indices_keyword(self,indices_keywords: Sequence[tuple], summary: Sequence[int] ,sent_num: int) -> torch.Tensor:

        tens_idx = torch.tensor([item[sent_num] for item in indices_keywords]).to(self.device)
        return tens_idx + summary

Trainer class that will handle the training phase for the RNN classifier

In [ ]:
class TaskBTrainer():
    def __init__(self, model, optimizer, device, exp_details):

        self.device = device
        self.model = model
        self.optimizer = optimizer
        self.writer = SummaryWriter(comment="_"+exp_details)
        self.model.train()  # we are using this model for training
        self.model.to(self.device)  # move model to GPU if available

    def train(self, train_dataset, eval_dataset, epochs: int = 1, early_stopping: bool = False, early_stopping_patience:int = 3, to_be_saved: bool =False) -> float:

        train_loss = 0.0
       
        for epoch in tqdm(range(epochs)):
            epoch_loss = 0.0
            len_train = 0
            
            self.model.train()

            # each element (sample) in train_dataset is a batch
            for sample in train_dataset:
                # inputs in the batch
                inputs = sample[0].to(self.device)
                # indices of keywords
                idx_start = sample[1].to(self.device)
                # outputs in the batch
                targets = sample[2].to(self.device)

                forward_result = self.model(inputs, idx_start, targets)

                loss = forward_result['loss']
                
                loss.backward()  #  backpropagate the loss
                # updates the parameters
                #Clips gradient norm of an iterable of parameters.
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), CLIP_GRAD)
                self.optimizer.step()
                self.optimizer.zero_grad()
                
                epoch_loss += loss.item()
                len_train += 1
            avg_epoch_loss = epoch_loss / len_train

            
            train_loss += avg_epoch_loss


            print(avg_epoch_loss)
            self.writer.flush()
    
        return train_loss/(epoch+1)

Loading of the handler for the dataset and choose of the batch size

In [ ]:
BATCH_SIZE = 40 #@param {type:"slider", min:8, max:64, step:4}

sentences_rnn_dm = TaskBDataModule(
    data_train_path=dataset_folder+'/laptops_train.json',
    data_dev_path=dataset_folder+'/laptops_dev.json',
    batch_size=BATCH_SIZE,
    collate_fn = rnn_collate_fn,
)

Hyperparameter setup

In [ ]:
#@title Setup of Hyper-parameters{ run: "auto" }

n_hidden=82 #@param {type:"slider", min:50, max:300, step:16}
drop_prob=0.15 #@param {type:"slider", min:0, max:1, step:0.05}
bidir = True #@param ["True", "False"] {type:"raw"}
learning_rate = 0.0001 #@param {type:"slider", min:0.00001, max:0.001, step:0.00001}
epochs = 25 #@param {type:"slider", min:10, max:100, step:10}
n_layer_lstm = 2 #@param {type:"slider", min:1, max:4, step:1}
CLIP_GRAD = 2 #@param {type:"slider", min:1, max:10, step:1}


Start the training

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#loading of the datasets
train_dataloader = sentences_rnn_dm.train_dataloader()
val_dataloader = sentences_rnn_dm.val_dataloader()


task_b_classifier = TaskBRecurrentClassifier(sentences_rnn_dm.train_dataset.vocabulary.vectors, n_hidden=n_hidden,drop_prob=drop_prob, bidir = bidir, n_layer_lstm = n_layer_lstm, vocab_size=len(sentences_rnn_dm.train_dataset.vocabulary),embedding_dim=100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#define the optimizer
optimizer = torch.optim.Adam(task_b_classifier.parameters(), lr=learning_rate)

# string to indetify the model once saved or in the graphs
exp_details="mul_leakyrelu_" + str(drop_prob) + "drop_"+str(n_hidden) +"hidden_"+str(learning_rate) +"lr_" + str(BATCH_SIZE) +"batch_" + str(n_layer_lstm) +"lstmLayer_" + str(CLIP_GRAD) +"clipGrad"

trainer = TaskBTrainer(task_b_classifier, optimizer, device, exp_details)

avg_train_loss= trainer.train(train_dataloader, val_dataloader, epochs=epochs, early_stopping=False, early_stopping_patience=6, to_be_saved = True)
print(" avg_train_loss={}\n ".format(avg_train_loss))



.vector_cache/glove.6B.zip: 862MB [02:41, 5.34MB/s]                           
  4%|▍         | 1/25 [00:01<00:26,  1.10s/it]

0.672441158765628


  8%|▊         | 2/25 [00:01<00:22,  1.02it/s]

0.4262655292764122


 12%|█▏        | 3/25 [00:02<00:19,  1.12it/s]

...

0.22823590251766604


 96%|█████████▌| 24/25 [00:17<00:00,  1.42it/s]

0.22627398702833387


100%|██████████| 25/25 [00:18<00:00,  1.38it/s]

0.22409910883432554
 avg_train_loss=0.264120972936536
 


## Evaluation

In [ ]:
 def eval_metrics(eval_dataset):
    task_b_classifier.eval()
    task_b_classifier.cuda()
    pred_outs, true_labels = [],[]
    for sample in eval_dataset:
        # inputs in the batch
        inputs = sample[0].to(device)
        idx_start = sample[1].to(device)

        # outputs in the batch
        targets = sample[2].to(device)
        with torch.no_grad():

            forward_result = task_b_classifier(inputs, idx_start,targets)
            loss = forward_result['loss']
            pred_out = torch.sigmoid(forward_result['logits'])
        
            pred_out = pred_out.detach().cpu().numpy()
            y_true = targets.cpu().numpy()

        pred_outs.append(pred_out)
        true_labels.append(y_true)

    flat_pred_outs = np.concatenate(pred_outs, axis=0)

    # Combine the correct labels for each batch into a single list.
    flat_true_labels = np.concatenate(true_labels, axis=0)

    return flat_pred_outs,flat_true_labels

In [ ]:
flat_pred_outs,flat_true_labels= eval_metrics(val_dataloader)
y_true = flat_true_labels.ravel()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
def classify(pred_prob):
    y_pred = []
    for tag_label_row in pred_prob:
        temp=[]
        max_prob = max(tag_label_row)
        for tag_label in tag_label_row:
            if tag_label == max_prob:
                temp.append(1)
            else:
                temp.append(0) 
        y_pred.append(temp)
    return y_pred

In [ ]:
y_pred_labels = classify(flat_pred_outs)

y_pred = np.array(y_pred_labels).ravel() # Flatten

metr = {}
metr['f1'] = f1_score(y_true,y_pred, average="macro")

print(metr['f1'])

0.7940042075736324


In [ ]:
y_pred = sentences_rnn_dm.validation_dataset.mlb.inverse_transform(np.array(y_pred_labels))
y_act = sentences_rnn_dm.validation_dataset.mlb.inverse_transform(flat_true_labels)

df = pd.DataFrame({'Body':sentences_rnn_dm.validation_dataset.sentences,'Actual Tags':y_act,'Predicted Tags':y_pred})
pd.set_option('display.max_rows', 500)
df.head(30)

,Body,Actual Tags,Predicted Tags
0,wa rated,"(NONE,)","(NONE,)"
1,Sony said could send back charged START adding...,"(negative,)","(negative,)"
2,START Windows 7 Starter END opinion great way ...,"(positive,)","(positive,)"
3,START powerpoint END opened seamlessly apple m...,"(positive,)","(positive,)"
4,chose iBookG4 laptop attractive computer large...,"(positive,)","(negative,)"
5,Finally biggest problem ha START tech support END,"(negative,)","(positive,)"
6,pleased purchase,"(NONE,)","(NONE,)"
7,Mac computer automatically defrag time start c...,"(NONE,)","(NONE,)"
8,Toshiba START sound END everything certain things,"(negative,)","(positive,)"
9,sleek lightweight START charge END quickly needed,"(positive,)","(negative,)"


## docker

In [ ]:
def load_data(data_path):
    with open(data_path) as json_file:
        list_of_sentences = json.load(json_file)
    return list_of_sentences

In [ ]:
def evaluate_sentiment(samples, predictions_b, mode="Aspect Sentiment"):
    scores = {}
    if mode == 'Category Extraction':
        sentiment_types = ["anecdotes/miscellaneous", "price", "food", "ambience"]
    else:
        sentiment_types = ["positive", "negative", "neutral", "conflict"]
    scores = {sent: {"tp": 0, "fp": 0, "fn": 0} for sent in sentiment_types + ["ALL"]}
    for label, pred in zip(samples, predictions_b):
        for sentiment in sentiment_types:
            if mode == "Aspect Sentiment":
                pred_sent = {(term_pred[0], term_pred[1]) for term_pred in pred["targets"] if
                                    term_pred[1] == sentiment}
                gt_sent = {(term_pred[1], term_pred[2]) for term_pred in label["targets"] if
                                    term_pred[2] == sentiment}
            elif mode == 'Category Extraction' and "categories" in label:
                pred_sent = {(term_pred[0]) for term_pred in pred["categories"] if
                                term_pred[0] == sentiment}
                gt_sent = {(term_pred[0]) for term_pred in label["categories"] if
                                term_pred[0] == sentiment}
            elif "categories" in label:
                pred_sent = {(term_pred[0], term_pred[1]) for term_pred in pred["categories"] if
                                term_pred[1] == sentiment}
                gt_sent = {(term_pred[0], term_pred[1]) for term_pred in label["categories"] if
                                term_pred[1] == sentiment}
            else:
                continue

            scores[sentiment]["tp"] += len(pred_sent & gt_sent)
            scores[sentiment]["fp"] += len(pred_sent - gt_sent)
            scores[sentiment]["fn"] += len(gt_sent - pred_sent)

    # Compute per sentiment Precision / Recall / F1
    for sent_type in scores.keys():
        if scores[sent_type]["tp"]:
            scores[sent_type]["p"] = 100 * scores[sent_type]["tp"] / (scores[sent_type]["fp"] + scores[sent_type]["tp"])
            scores[sent_type]["r"] = 100 * scores[sent_type]["tp"] / (scores[sent_type]["fn"] + scores[sent_type]["tp"])
        else:
            scores[sent_type]["p"], scores[sent_type]["r"] = 0, 0

        if not scores[sent_type]["p"] + scores[sent_type]["r"] == 0:
            scores[sent_type]["f1"] = 2 * scores[sent_type]["p"] * scores[sent_type]["r"] / (
                    scores[sent_type]["p"] + scores[sent_type]["r"])
        else:
            scores[sent_type]["f1"] = 0

    # Compute micro F1 Scores
    tp = sum([scores[sent_type]["tp"] for sent_type in sentiment_types])
    fp = sum([scores[sent_type]["fp"] for sent_type in sentiment_types])
    fn = sum([scores[sent_type]["fn"] for sent_type in sentiment_types])

    if tp:
        precision = 100 * tp / (tp + fp)
        recall = 100 * tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)

    else:
        precision, recall, f1 = 0, 0, 0

    scores["ALL"]["p"] = precision
    scores["ALL"]["r"] = recall
    scores["ALL"]["f1"] = f1
    scores["ALL"]["tp"] = tp
    scores["ALL"]["fp"] = fp
    scores["ALL"]["fn"] = fn

    # Compute Macro F1 Scores
    scores["ALL"]["Macro_f1"] = sum([scores[ent_type]["f1"] for ent_type in sentiment_types])/len(sentiment_types)
    scores["ALL"]["Macro_p"] = sum([scores[ent_type]["p"] for ent_type in sentiment_types])/len(sentiment_types)
    scores["ALL"]["Macro_r"] = sum([scores[ent_type]["r"] for ent_type in sentiment_types])/len(sentiment_types)

    print(f"{mode} Evaluation\n")

    print(
        "\tALL\t TP: {};\tFP: {};\tFN: {}".format(
            scores["ALL"]["tp"],
            scores["ALL"]["fp"],
            scores["ALL"]["fn"]))
    print(
        "\t\t(m avg): precision: {:.2f};\trecall: {:.2f};\tf1: {:.2f} (micro)".format(
            precision,
            recall,
            f1))
    print(
        "\t\t(M avg): precision: {:.2f};\trecall: {:.2f};\tf1: {:.2f} (Macro)\n".format(
            scores["ALL"]["Macro_p"],
            scores["ALL"]["Macro_r"],
            scores["ALL"]["Macro_f1"]))

    for sent_type in sentiment_types:
        print("\t{}: \tTP: {};\tFP: {};\tFN: {};\tprecision: {:.2f};\trecall: {:.2f};\tf1: {:.2f};\t{}".format(
            sent_type,
            scores[sent_type]["tp"],
            scores[sent_type]["fp"],
            scores[sent_type]["fn"],
            scores[sent_type]["p"],
            scores[sent_type]["r"],
            scores[sent_type]["f1"],
            scores[sent_type]["tp"] +
            scores[sent_type][
                "fp"]))

In [ ]:
class PreprocessBNEW():
    def __init__(self, sentences):
        self.data_store,self.sentences,self.targets = self.load_data(sentences)
        self.vocabulary = self.build_vocab(self.sentences)
        self.convert_all_2_indices()
        self.tags = [["neutral","negative", "conflict", "positive", "NONE"]]
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit_transform(self.tags)
        
    def remove_stopwords(self,sent: str) -> str:
        stop_words = set(stopwords.words('english'))

        # remove punkt
        others = "–" +"—" + "−" + "’" + "”" + "“" #These chars arent inside the standard punctuation
        str_punkt = string.punctuation+ others
        translator = str.maketrans(str_punkt, ' '*len(str_punkt)) 
        word_tokens = word_tokenize(sent.translate(translator)) 
        
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        return filtered_sentence

    def load_data(self,list_of_sentences):
        data_store,sentences,targets = [],[],[]
        for obj in list_of_sentences:
            _sentence = []
            obj['targets'] = sorted(obj['targets'], key=lambda x: x[0][0])
            sentiments = [obj['targets'][j][2] for j in range(len(obj['targets']))]

            for i,targ_obj in enumerate(obj['targets']):
                #print(targ_obj)
                new_sent = obj['text'][:targ_obj[0][0]-1]+" <START> " + obj['text'][targ_obj[0][0]:targ_obj[0][1]] + " <END>" + obj['text'][targ_obj[0][1]:]
                new_sent = [lemmatizer.lemmatize(w)  for w in new_sent.split(" ")]
                new_sent = " ".join(self.remove_stopwords(" ".join(new_sent)))    
                index = self.find_indices(new_sent)
                    
                sentences.append(new_sent)
                sentiments_converted = [sentiments[i]]
                targets.append([(targ[1], "") for j,targ in enumerate(obj['targets'])])

                data_store.append((new_sent,torch.tensor(index,dtype=torch.long), sentiments_converted))

            if len(obj['targets'])==0:
                sentiments_converted= ["NONE"]
                targets.append([(targ[1], "") for j,targ in enumerate(obj['targets'])])

                new_sent = obj['text']
                # new_sent = [lemmatizer.lemmatize(w)  for w in new_sent.split(" ")]
                # new_sent = " ".join(self.remove_stopwords(" ".join(new_sent)))
                index = [0,0]
                sentences.append(new_sent)
                data_store.append((new_sent,torch.tensor(index,dtype=torch.long), sentiments_converted))
        
        return data_store,sentences,targets


    def convert_all_2_indices(self):
        self.data_store = [(self.sentence2indices(a),b,c) for (a,b,c) in self.data_store]
    
    def find_indices(self,new_sent):
        splitted = new_sent.split(" ")
        indices = [i+1 for i,w in enumerate(splitted) if (w=="START") or (w=="END")]
        indices[1] = indices[1]-1
        return indices
                
    
    def build_vocab(self, dataset, min_freq=1):
        counter = Counter()
        for a in dataset:
            # for each token in the sentence viewed as a dictionary of items from the line
            for token in a.split(" "):
                if token is not None:
                    counter[token]+=1
        #Counter({'the': 1046, 'and': 671, 'to': 604, etc....}
        # we add special tokens for handling padding and unknown words at testing time.
        vocabulary = Vocab(counter, specials=['<pad>', '<unk>'], min_freq=min_freq)
        #vocabulary.load_vectors("glove.6B.50d")
        return vocabulary


    def __len__(self) -> int:
        return len(self.sentences)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.data_store[idx]

    def sentence2indices(self,sentence: str) -> torch.Tensor:
        return torch.tensor([self.vocabulary[word] for word in sentence.split(' ') if word != ''], dtype=torch.long)

    def decode_tag(self,tag):
        return self.mlb.inverse_transform(tag.reshape(1,-1))
    
    
def rnn_collate_fn(data_elements: List[Tuple[torch.Tensor, list]] # list of (x, y,z) pairs
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    X = [de[0] for de in data_elements]  # list of index tensors
    X = torch.nn.utils.rnn.pad_sequence(X, batch_first=True, padding_value=0)  #  shape (batch_size x max_seq_len)
    
    keyword_position = [de[1] for de in data_elements] # list of tuples indices where keyword is [[1st sent, 2nd sent]]

    keyword_position = torch.nn.utils.rnn.pad_sequence(keyword_position, batch_first=True, padding_value=0) 



    return X, keyword_position

def classify(pred_prob):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        max_prob = max(tag_label_row)
        for tag_label in tag_label_row:
            if tag_label == max_prob:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred
    
# def predictBNEW(samples: List[Dict]) -> List[Dict]:
#     targets = []
#     prep = PreprocessBNEW(samples)
#     task_b_classifier.eval()
#     pred_outs = []
#     for i,sample in enumerate(prep.data_store):
#         try: #yet started
#             started = started
#         except:
#             started = False
        
#         if len(prep.targets[i])>1 and not started:
#             cont = len(prep.targets[i])
#             started = True
#             json_pred = {"targets":prep.targets[i]}
#             sentiments = []
        
#         elif len(prep.targets[i]) == 1:
#             json_pred = {"targets":prep.targets[i]}
#             cont = 1
#             sentiments = []
#         elif len(prep.targets[i]) == 0:
#             cont = 0
#             json_pred = {"targets":prep.targets[i]}
#             sentiments = []
        
#         lst,pred = [],[]

#         inputs,idx_start = rnn_collate_fn([sample]) # inputs in the batch
#         with torch.no_grad():
#             forward_result = task_b_classifier(inputs.cpu(), idx_start.cpu())
#             pred_out = torch.sigmoid(forward_result['logits'])
#             pred_out = pred_out.detach().cpu().numpy()

#         y_pred_labels = classify(pred_out)
#         y_pred = prep.mlb.inverse_transform(np.array(y_pred_labels))
#         for pred in y_pred:
#             sentiments += list(pred)
        
#         cont-=1
#         if cont <= 0:
#             for k,targ in enumerate(json_pred['targets']):
#                 try:
#                     if sentiments[k] != "NONE":
#                         json_pred["targets"][k] = (json_pred["targets"][k][0],sentiments[k])
#                 except:
#                     json_pred["targets"][k] = (json_pred["targets"][k][0],"conflict")
#             started = False
#             if len(prep.targets[i]) == len(json_pred['targets']):
#                 targets.append(json_pred)
#             else:
#                 print(prep.sentences[i-1], prep.targets[i-1],targets[-2]['targets'])
#                 print(prep.sentences[i], prep.targets[i] ,json_pred['targets'])
#                 targets[-2]['targets'] = json_pred['targets']
#                 targets.append(json_pred)
#     return targets

def predictBBNEW(samples: List[Dict]) -> List[Dict]:
    targets = []
    prep = PreprocessBNEW(samples)
    task_b_classifier.eval()
    i = 0
    task_b_classifier.cpu()
    print(len(prep.data_store))
    while i < len(prep.data_store):
        cont = len(prep.targets[i])
        json_pred = {"targets":prep.targets[i]}
        if cont==0:
            i+=1
        sentiments = []
        while cont > 0:
            inputs,idx_start = rnn_collate_fn([prep.data_store[i]]) # inputs in the batch
            
            with torch.no_grad():
                forward_result = task_b_classifier(inputs.cpu(), idx_start.cpu())
                pred_out = torch.sigmoid(forward_result['logits'])
                pred_out = pred_out.detach().cpu().numpy()

            y_pred_labels = classify(pred_out)
            y_pred = prep.mlb.inverse_transform(np.array(y_pred_labels))
            for pred in y_pred:
                sentiments += list(pred)
            cont-=1
            i+=1
        for k,targ in enumerate(json_pred['targets']):
            try:
                if sentiments[k] != "NONE":
                    json_pred["targets"][k] = (json_pred["targets"][k][0],sentiments[k])
            except:
                json_pred["targets"][k] = (json_pred["targets"][k][0],"conflict")
        targets.append(json_pred)
    return targets

a = load_data(dataset_folder+"/laptops_dev.json")
random.shuffle(a)
t = predictBBNEW(a)
for x,y in zip(a[:30],t[:30]):
    print(x['targets'],"SEP",y['targets'])

713
[[[131, 143], 'applications', 'neutral']] SEP [('applications', 'positive')]
[] SEP []
[[[11, 19], 'programs', 'positive'], [[21, 28], 'Keynote', 'neutral'], [[30, 35], 'Pages', 'neutral'], [[37, 44], 'Numbers', 'neutral']] SEP [('programs', 'positive'), ('Keynote', 'positive'), ('Pages', 'positive'), ('Numbers', 'positive')]
[[[21, 29], 'operates', 'positive']] SEP [('operates', 'positive')]
[[[0, 8], 'Delivery', 'positive']] SEP [('Delivery', 'positive')]
[] SEP []
[] SEP []
[[[110, 117], 'company', 'negative']] SEP [('company', 'positive')]
[[[12, 25], 'aluminum body', 'positive']] SEP [('aluminum body', 'positive')]
[[[0, 14], 'Windows XP SP2', 'negative']] SEP [('Windows XP SP2', 'positive')]
[] SEP []
[[[10, 29], 'Toshiba online help', 'positive']] SEP [('Toshiba online help', 'positive')]
[[[32, 39], 'charges', 'positive']] SEP [('charges', 'positive')]
[[[37, 41], 'look', 'positive'], [[61, 80], 'after sales support', 'negative']] SEP [('look', 'negative'), ('after sales su

In [ ]:
evaluate_sentiment(a,t)

Aspect Sentiment Evaluation

	ALL	 TP: 195;	FP: 229;	FN: 231
		(m avg): precision: 45.99;	recall: 45.77;	f1: 45.88 (micro)
		(M avg): precision: 22.78;	recall: 27.85;	f1: 22.94 (Macro)

	positive: 	TP: 160;	FP: 186;	FN: 23;	precision: 46.24;	recall: 87.43;	f1: 60.49;	346
	negative: 	TP: 35;	FP: 43;	FN: 111;	precision: 44.87;	recall: 23.97;	f1: 31.25;	78
	neutral: 	TP: 0;	FP: 0;	FN: 91;	precision: 0.00;	recall: 0.00;	f1: 0.00;	0
	conflict: 	TP: 0;	FP: 0;	FN: 6;	precision: 0.00;	recall: 0.00;	f1: 0.00;	0


# TASK A-B (DistillBERT fine tuning)

## Dataset

Define of the transfomers dataset class, this class will load raw data and then encode the labels into tag and using the Bert tokenizer, tokenize the sentences.

In [ ]:
class TransformersDataset(Dataset):
    def __init__(self, path,path2, model_b):
        self.model_b = model_b
        self.texts, self.tags = self.load_data(path)
        texts2,tags2 = self.load_data(path2)
        self.texts = self.texts + texts2
        self.tags = self.tags + tags2

        self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
        self.encodings = self.tokenizer(self.texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
        self.unique_tags = set(tag for doc in self.tags for tag in doc)
        self.tag2id = {tag: id for id, tag in enumerate(self.unique_tags)}
        self.id2tag = {id: tag for tag, id in self.tag2id.items()}
        self.labels = self.encode_tags(self.tags, self.encodings)
        

    def load_data(self,data_path):
        sentences,texts,tags = [], [], []
        with open(data_path) as json_file:
            list_of_sentences = json.load(json_file)
            for obj in list_of_sentences:
                _sentence = []
                for t in tokenizer.tokenize(obj['text']):
                    ne_label = "O"
                    sentiment = ""
                    for i in range(len(obj['targets'])):
                        if t in obj['targets'][i][1] and len(t)>2:
                            #if its first word i.e. "battery life" (battery)
                            ne_label = "B" if t == obj['targets'][i][1].split(" ")[0] else "I"
                            #if for any reason the I tag is assigned but the word before its not a B
                            try:
                                if ne_label == "I" and _sentence[-1]["ne_label"] == "O":
                                    ne_label = "O"
                            except:
                                pass
                            sentiment = obj['targets'][i][2]
                            if self.model_b and ne_label != "O":
                                ne_label = ne_label +"-"+sentiment
                    token = {"token": t, "ne_label": ne_label , "sentiment" :sentiment}
                    _sentence.append(token)
                sentences.append(_sentence)

            for elem in sentences:
                texts.append([tok['token'] for tok in elem])
                tags.append([tag['ne_label'] for tag in elem])
            return texts, tags
    
    def encode_tags(self,tags, encodings):
        labels = [[self.tag2id[tag] for tag in doc] for doc in tags]
        encoded_labels = []
        for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
            # create an empty array of -100
            doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
            arr_offset = np.array(doc_offset)

            try:
                # set labels whose first offset position is 0 and the second is not 0
                doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
                encoded_labels.append(doc_enc_labels.tolist())
            except:
                print(doc_labels, doc_offset)

        return encoded_labels


    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['text'] = self.texts[idx] + ["<end>"] *(100-len(self.texts[idx]))
        return item

    def __len__(self):
        return len(self.labels)


## Training

I used a custom Trainer, instead of using the one of huggingface because I do not know if we were allowed to.

In [ ]:
class TrainerTransformers():
    def __init__(self, model, optimizer, device,comments="transformers"):

        self.device = device
        self.model = model
        self.optimizer = optimizer
        self.writer = SummaryWriter(comment="_"+comments)
        self.model.train()  # we are using this model for training
        self.model.to(self.device)  # move model to GPU if available

    def train(self, train_loader, val_loader, epochs: int = 1) -> float:
        for epoch in tqdm(range(epochs)):
            tot_loss,tot_train_f1 = 0,0
            model.train()
            for i,batch in enumerate(train_loader):
                optim.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs[0]
                loss.backward()
                optim.step()
                tot_loss += loss.item()
                #tot_train_f1+=self.calc_f1(outputs['logits'],labels,batch['text'])
                
            train_loss,train_f1 = tot_loss/i+1, tot_train_f1/i+1
            val_loss,val_f1 = self.eval_metrics(val_loader,epochs)
            
            self.writer.add_scalar("train/loss", train_loss, epoch)
            #self.writer.add_scalar("train/f1", tot_train_f1, epoch)

            self.writer.add_scalar("eval/loss", val_loss, epoch)
            #self.writer.add_scalar("eval/f1", val_f1, epoch)
            
            print("epoch {} train_loss={:0.4f} eval_loss={:0.4f}".format(epoch+1,train_loss,val_loss))

    def eval_metrics(self,val_loader,epochs):
        model.eval()
        tot_val_loss,tot_val_f1 = 0,0
        for idx,batch in enumerate(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            #tot_val_f1+=self.calc_f1(outputs['logits'],labels,batch['text'])

            tot_val_loss += loss.item()
        return tot_val_loss/idx+1, tot_val_f1/idx+1

    
    # def calc_f1(self,logits,labels,text):
    #     lst = []
    #     for id in logits[0].argmax(1):
    #         lst.append(id.item())
    #     ground_truth = labels
    #     text = [sent for sent in text if "<end>" not in sent]
    #     text = [item for sublist in text for item in sublist]
    #     print(text)
    #     pred = self.new_logits(text, lst,val_dataset.tokenizer)[1:-1]
    #     print(pred)

    #     f1_macro = f1_score(ground_truth, pred, average="macro", zero_division=0)
    #     return f1_macro
    # def new_logits(self,text,logits,tokenizer):
    #     new_logits = list()
    #     for t in text:
    #         offset = tokenizer(t, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)['offset_mapping']
    #         for i, tup in enumerate(offset):
    #             if tup[0] == 0:
    #                 new_logits.append(logits[i])
    #     return new_logits

Declaration of the datasets, dataloaders and optimizer

In [ ]:
train_dataset = TransformersDataset(dataset_folder+"/laptops_train.json",dataset_folder+"/restaurants_train.json",model_b)
val_dataset = TransformersDataset(dataset_folder+"/laptops_dev.json",dataset_folder+"/restaurants_dev.json",model_b)

In [ ]:
for id, tag in enumerate(val_dataset.unique_tags):
    print(id,tag)

0 I-conflict
1 I-negative
2 I-neutral
3 B-positive
4 B-conflict
5 B-negative
6 O
7 B-neutral
8 I-positive


In [ ]:
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(train_dataset.unique_tags))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optim = AdamW(model.parameters(), lr=5e-5)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

Start of the training

In [ ]:
trainer = TrainerTransformers(model,optim,device, comments="comment")
trainer.train(train_loader,val_loader, epochs=5)

 33%|███▎      | 1/3 [00:27<00:55, 27.76s/it]

epoch 1 train_loss=1.2462 eval_loss=1.1778


 67%|██████▋   | 2/3 [00:55<00:27, 27.77s/it]

epoch 2 train_loss=1.1232 eval_loss=1.1656


100%|██████████| 3/3 [01:23<00:00, 27.77s/it]

epoch 3 train_loss=1.0698 eval_loss=1.1941


## Evaluation

In [ ]:
def new_logits(text,logits,tokenizer):
    offset = tokenizer(text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)['offset_mapping']
    new_logits = list()
    for i, tup in enumerate(offset):
        if tup[0] == 0:
            new_logits.append(logits[i])
    return new_logits

def calculate_metrics(model,val_dataset):
    model.eval()
    model.cuda()
    ground_truth,pred = [],[]
    for i,t in enumerate(val_dataset.texts):
        lst = []
        ids = val_dataset.tokenizer(val_dataset.texts[i], is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)["input_ids"]
        logits = model(torch.tensor([ids]).to(device))["logits"]
        for id in logits[0].argmax(1):
            lst.append(id.item())
        #print(val_dataset.texts[i])
        ground_truth.append([val_dataset.tag2id[tag] for tag in val_dataset.tags[i]])
        pred.append((new_logits(val_dataset.texts[i], lst,val_dataset.tokenizer)[1:-1]))
    ground_truth = [item for sublist in ground_truth for item in sublist]
    pred = [item for sublist in pred for item in sublist]
    
    micro_precision = sk_precision(ground_truth, pred, average="macro", zero_division=0)
    
    # precision per class and arithmetic average of them. Does not take into account class imbalance.
    f1_macro = f1_score(ground_truth, pred, average="macro", zero_division=0)
    per_class_precision = sk_precision(ground_truth, pred, labels = list(range(len(val_dataset.tag2id))), average=None, zero_division=0)
    precisions = {"micro_precision":micro_precision, "f1_macro":f1_macro, "per_class_precision":per_class_precision}
    return precisions

precisions = calculate_metrics(model,val_dataset)

print("Macro Precision: {}\nMacro F1: {}".format(precisions["micro_precision"], precisions["f1_macro"]))
print("Per class Precision:")
for idx_class, precision in sorted(enumerate(precisions["per_class_precision"]), key=lambda elem: -elem[1]):
    label = val_dataset.id2tag[idx_class]
    print(label, precision)

Macro Precision: 0.44363683375640456
Macro F1: 0.44115863159254076
Per class Precision:
O 0.9870322080484882
B-positive 0.6145833333333334
B-negative 0.5494505494505495
I-positive 0.525096525096525
I-negative 0.48514851485148514
B-neutral 0.4411764705882353
I-neutral 0.3902439024390244
I-conflict 0.0
B-conflict 0.0


In [ ]:
torch.save(model.state_dict(), root_folder+'/model/model_AB_together={}_f1_{:0.4f}.pt'.format(str(model_b), precisions["f1_macro"])) # save the model state

## docker

#### Model B

In [ ]:
class PreprocessB():
    def __init__(self, sentences):
        self.texts, self.tags = self.load_data(sentences)
        self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
        self.encodings = self.tokenizer(self.texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
        
        self.tag2id = {'neutral':0, 'negative':1, 'conflict':2, 'O':3,'positive':4 }
        #self.tag2id = {tag: id for id, tag in enumerate(self.unique_tags)}
        self.id2tag = {id: tag for tag, id in self.tag2id.items()}
                

    def load_data(self,list_of_sentences):
        sentences,texts,tags = [], [], []
        for obj in list_of_sentences:
            _sentence = []
            for t in tokenizer.tokenize(obj['text']):
                ne_label ="O"
                for i in range(len(obj['targets'])):
                    if t in obj['targets'][i][1]: #if target word
                        ne_label = obj['targets'][i][1]
                        break
                        
                token = {"token": t, "ne_label": ne_label}
                _sentence.append(token)

            sentences.append(_sentence)

        for i,elem in enumerate(sentences):
            texts.append([tok['token'] for tok in elem])
            #new_lst = list()
            tags.append([(targ[1], "") for j,targ in enumerate(list_of_sentences[i]['targets'])])
            #print(tags)
            # for tag in elem:
            #     if tag['ne_label'] != "O" and tag['ne_label'] not in new_lst:
            #         new_lst.append(tag['ne_label'])
            # tags.append(new_lst)
        return texts, tags
    

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def new_logits(text,logits,tokenizer):
    offset = tokenizer(text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)['offset_mapping']
    new_logits = list()
    for i, tup in enumerate(offset):
        if tup[0] == 0:
            new_logits.append(logits[i])
    return new_logits

def predictB(samples: List[Dict]) -> List[Dict]:
    targets = []
    prep = PreprocessB(samples)
    for i,encode in enumerate(prep.encodings['input_ids']):
        json_pred = {"targets":prep.tags[i]}
        lst,pred = [],[]
        ids = torch.unsqueeze(torch.tensor(encode),0)
        attention_mask = torch.unsqueeze(torch.tensor(prep.encodings['attention_mask'][i]),0)

        logits = model(torch.tensor(ids).to("cpu"), torch.tensor(attention_mask).cpu())["logits"]
        for id in logits[0].argmax(1):
            lst.append(id.item())
        p = new_logits(prep.texts[i], lst,prep.tokenizer)[1:-1]
        idtotag = [prep.id2tag[raw_pred] for raw_pred in p]
        sentiments = []
        # for j,sentiment in enumerate(idtotag):
        #     if (sentiment != "O"):
        #         try:
        #             if (idtotag[j+1] == "O"):
        #                 sentiments.append(sentiment)
        #                 #print(prep.texts[i][j],sentiment)
        #         except:
        #             sentiments.append(sentiment)
        #             #print(prep.texts[i][j],sentiment)

                
        # for k,targ in enumerate(json_pred['targets']):
        #     try:
        #         json_pred["targets"][k] = (json_pred["targets"][k][0],sentiments[k])
        #     except:
        #         print(prep.texts[i],idtotag)
        #         json_pred["targets"][k] = (json_pred["targets"][k][0],"conflict")
        idd = 0
        if (len(json_pred['targets'])>0):
            for word, sentiment in zip(prep.texts[i],idtotag):
                if word in json_pred["targets"][idd][0]:
                    if sentiment !="O": 
                        json_pred["targets"][idd] = (json_pred["targets"][idd][0],sentiment)
                    else:
                        json_pred["targets"][idd] = (json_pred["targets"][idd][0],"positive") 
                    idd+=1
                    if idd==len(json_pred["targets"]):
                        break
        targets.append(json_pred)
    return targets

###Model AB

In [ ]:
class PreprocessAB():
    def __init__(self, sentences):
        self.texts = self.load_data(sentences)
        self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
        self.encodings = self.tokenizer(self.texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
        
        self.tag2id = {'O':0, 'I':1, 'B':2}
        #self.tag2id = {tag: id for id, tag in enumerate(self.unique_tags)}
        self.id2tag = {id: tag for tag, id in self.tag2id.items()}
                

    def load_data(self,list_of_sentences):
        sentences,texts = [], []
        for obj in list_of_sentences:
            _sentence = []
            for t in tokenizer.tokenize(obj['text']):
                token = {"token": t}
                _sentence.append(token)
            sentences.append(_sentence)

        for elem in sentences:
            texts.append([tok['token'] for tok in elem])
        return texts
    
    

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def predictAB(samples: List[Dict]) -> List[Dict]:
    targets = []
    prep = PreprocessAB(samples)

    for i,encode in tqdm(enumerate(prep.encodings['input_ids'])):
        json_pred = {"targets":[]}
        lst,pred = [],[]
        ids = torch.unsqueeze(torch.tensor(encode),0)
        attention_mask = torch.unsqueeze(torch.tensor(prep.encodings['attention_mask'][i]),0)
        logits = modelAB(torch.tensor(ids).to("cpu"), torch.tensor(attention_mask).cpu())["logits"]
        
        for id in logits[0].argmax(1):
            lst.append(id.item())
        p = new_logits(prep.texts[i], lst,prep.tokenizer)[1:-1]
        idtotag = [prep.id2tag[raw_pred] for raw_pred in p]

        for j,word in enumerate(idtotag):
            if word == "B":
                json_pred["targets"].append((prep.texts[i][j],"positive"))
            elif (word == "I") and (idtotag[j-1] == "B"):
                try:
                    last_tuple = json_pred['targets'][-1]
                    words_tagged = last_tuple[0] + " " + prep.texts[i][j]
                    sent_tagged = last_tuple[1]
                    json_pred['targets'][-1] = (words_tagged, sent_tagged)
                except:
                    words_tagged = prep.texts[i][j]
                    sent_tagged = "positive"
                    json_pred['targets'].append((words_tagged, sent_tagged))
            elif word == "I":
                json_pred["targets"].append((prep.texts[i][j],"positive"))

        targets.append(predictBAB(json_pred,prep.texts[i],prep.tokenizer,ids,attention_mask))

    return targets




def predictBAB(json_pred,text,tokenizer,ids,attention_mask) -> List[Dict]:
    tag2id = {'neutral':0, 'negative':1, 'conflict':2, 'O':3,'positive':4 }
    id2tag = {id: tag for tag, id in tag2id.items()}
    
    lst,pred = [],[]
    logits = modelB(torch.tensor(ids).to("cpu"), torch.tensor(attention_mask).cpu())["logits"]
    for id in logits[0].argmax(1):
        lst.append(id.item())
    p = new_logits(text, lst,tokenizer)[1:-1]
    idtotag = [id2tag[raw_pred] for raw_pred in p]
    sentiments = []
    
    idd = 0
    if (len(json_pred['targets'])>0):
        for word, sentiment in zip(text,idtotag):
            if word in json_pred["targets"][idd][0]:
                if sentiment !="O": 
                    json_pred["targets"][idd] = (json_pred["targets"][idd][0],sentiment)
                else:
                    json_pred["targets"][idd] = (json_pred["targets"][idd][0],"positive") 
                idd+=1
                if idd==len(json_pred["targets"]):
                    break
    
    return json_pred




def predict_together_AB(samples: List[Dict]) -> List[Dict]:
    targets = []
    prep = PreprocessAB(samples)
    for i,encode in tqdm(enumerate(prep.encodings['input_ids'])):
        json_pred = {"targets":[]}
        lst,pred = [],[]
        ids = torch.unsqueeze(torch.tensor(encode),0)
        attention_mask = torch.unsqueeze(torch.tensor(prep.encodings['attention_mask'][i]),0)
        logits = model(torch.tensor(ids).to("cpu"), torch.tensor(attention_mask).cpu())["logits"]
        
        for id in logits[0].argmax(1):
            lst.append(id.item())
        p = new_logits(prep.texts[i], lst,prep.tokenizer)[1:-1]
        #idtotag = [val_dataset.id2tag[raw_pred] for raw_pred in p]
        idtotag = [label_vocabulary.itos[raw_pred] for raw_pred in p]



        for j,word in enumerate(idtotag):
            if word.startswith("B"):
                json_pred["targets"].append((prep.texts[i][j],word.split("-")[1]))
            elif (word.startswith("I")) and (idtotag[j-1].startswith("B")):
                try:
                    last_tuple = json_pred['targets'][-1]
                    words_tagged = last_tuple[0] + " " + prep.texts[i][j]
                    sent_tagged = last_tuple[1]
                    json_pred['targets'][-1] = (words_tagged, sent_tagged)
                except:
                    words_tagged = prep.texts[i][j]
                    sent_tagged = word.split("-")[1]
                    json_pred['targets'].append((words_tagged, sent_tagged))
            elif word == "I":
                json_pred["targets"].append((prep.texts[i][j],word.split("-")[1]))

        targets.append(predictBAB(json_pred,prep.texts[i],prep.tokenizer,ids,attention_mask))

    return targets

### test

In [ ]:
def load_data(data_path):
    with open(data_path) as json_file:
        list_of_sentences = json.load(json_file)
    return list_of_sentences


# model_state_dictB = torch.load(root_folder+"/model/model_b=True_f1_0.5406.pt", map_location=torch.device("cpu"))
# modelB = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', state_dict=model_state_dictB, num_labels=5)
# modelB.eval()
# modelB.cpu()


# model_state_dictAB = torch.load(root_folder+"/model/model_b=False_f1_0.8333.pt", map_location=torch.device("cpu"))
# modelAB = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', state_dict=model_state_dictAB, num_labels=3)
# modelAB.eval()
# modelAB.cpu()

In [ ]:
a = load_data(dataset_folder+"/laptops_dev.json")
a = a + load_data(dataset_folder+"/restaurants_dev.json")
random.shuffle(a)
model.cpu()
#t = predictB(a[:30])
t = predict_together_AB(a)

In [ ]:
for gt,pred in zip(a[:40],t[:40]):
    print(gt['targets'],"SEP",pred['targets'])

In [ ]:
evaluate_sentiment(a,t)
evaluate_extraction(a,t)

Aspect Sentiment Evaluation

	ALL	 TP: 9;	FP: 1219;	FN: 1074
		(m avg): precision: 0.73;	recall: 0.83;	f1: 0.78 (micro)
		(M avg): precision: 0.66;	recall: 0.67;	f1: 0.65 (Macro)

	positive: 	TP: 4;	FP: 701;	FN: 536;	precision: 0.57;	recall: 0.74;	f1: 0.64;	705
	negative: 	TP: 3;	FP: 221;	FN: 299;	precision: 1.34;	recall: 0.99;	f1: 1.14;	224
	neutral: 	TP: 2;	FP: 267;	FN: 214;	precision: 0.74;	recall: 0.93;	f1: 0.82;	269
	conflict: 	TP: 0;	FP: 30;	FN: 25;	precision: 0.00;	recall: 0.00;	f1: 0.00;	30
Aspect Extraction Evaluation
	Aspects	 TP: 42;	FP: 1186;	FN: 1037
		precision: 3.42;	recall: 3.89;	f1: 3.64


In [41]:
def evaluate_sentiment(samples, predictions_b, mode="Aspect Sentiment"):
    scores = {}
    if mode == 'Category Extraction':
        sentiment_types = ["anecdotes/miscellaneous", "price", "food", "ambience"]
    else:
        sentiment_types = ["positive", "negative", "neutral", "conflict"]
    scores = {sent: {"tp": 0, "fp": 0, "fn": 0} for sent in sentiment_types + ["ALL"]}
    for label, pred in zip(samples, predictions_b):
        for sentiment in sentiment_types:
            if mode == "Aspect Sentiment":
                pred_sent = {(term_pred[0], term_pred[1]) for term_pred in pred["targets"] if
                                    term_pred[1] == sentiment}
                gt_sent = {(term_pred[1], term_pred[2]) for term_pred in label["targets"] if
                                    term_pred[2] == sentiment}
            elif mode == 'Category Extraction' and "categories" in label:
                pred_sent = {(term_pred[0]) for term_pred in pred["categories"] if
                                term_pred[0] == sentiment}
                gt_sent = {(term_pred[0]) for term_pred in label["categories"] if
                                term_pred[0] == sentiment}
            elif "categories" in label:
                pred_sent = {(term_pred[0], term_pred[1]) for term_pred in pred["categories"] if
                                term_pred[1] == sentiment}
                gt_sent = {(term_pred[0], term_pred[1]) for term_pred in label["categories"] if
                                term_pred[1] == sentiment}
            else:
                continue

            scores[sentiment]["tp"] += len(pred_sent & gt_sent)
            scores[sentiment]["fp"] += len(pred_sent - gt_sent)
            scores[sentiment]["fn"] += len(gt_sent - pred_sent)

    # Compute per sentiment Precision / Recall / F1
    for sent_type in scores.keys():
        if scores[sent_type]["tp"]:
            scores[sent_type]["p"] = 100 * scores[sent_type]["tp"] / (scores[sent_type]["fp"] + scores[sent_type]["tp"])
            scores[sent_type]["r"] = 100 * scores[sent_type]["tp"] / (scores[sent_type]["fn"] + scores[sent_type]["tp"])
        else:
            scores[sent_type]["p"], scores[sent_type]["r"] = 0, 0

        if not scores[sent_type]["p"] + scores[sent_type]["r"] == 0:
            scores[sent_type]["f1"] = 2 * scores[sent_type]["p"] * scores[sent_type]["r"] / (
                    scores[sent_type]["p"] + scores[sent_type]["r"])
        else:
            scores[sent_type]["f1"] = 0

    # Compute micro F1 Scores
    tp = sum([scores[sent_type]["tp"] for sent_type in sentiment_types])
    fp = sum([scores[sent_type]["fp"] for sent_type in sentiment_types])
    fn = sum([scores[sent_type]["fn"] for sent_type in sentiment_types])

    if tp:
        precision = 100 * tp / (tp + fp)
        recall = 100 * tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)

    else:
        precision, recall, f1 = 0, 0, 0

    scores["ALL"]["p"] = precision
    scores["ALL"]["r"] = recall
    scores["ALL"]["f1"] = f1
    scores["ALL"]["tp"] = tp
    scores["ALL"]["fp"] = fp
    scores["ALL"]["fn"] = fn

    # Compute Macro F1 Scores
    scores["ALL"]["Macro_f1"] = sum([scores[ent_type]["f1"] for ent_type in sentiment_types])/len(sentiment_types)
    scores["ALL"]["Macro_p"] = sum([scores[ent_type]["p"] for ent_type in sentiment_types])/len(sentiment_types)
    scores["ALL"]["Macro_r"] = sum([scores[ent_type]["r"] for ent_type in sentiment_types])/len(sentiment_types)

    print(f"{mode} Evaluation\n")

    print(
        "\tALL\t TP: {};\tFP: {};\tFN: {}".format(
            scores["ALL"]["tp"],
            scores["ALL"]["fp"],
            scores["ALL"]["fn"]))
    print(
        "\t\t(m avg): precision: {:.2f};\trecall: {:.2f};\tf1: {:.2f} (micro)".format(
            precision,
            recall,
            f1))
    print(
        "\t\t(M avg): precision: {:.2f};\trecall: {:.2f};\tf1: {:.2f} (Macro)\n".format(
            scores["ALL"]["Macro_p"],
            scores["ALL"]["Macro_r"],
            scores["ALL"]["Macro_f1"]))

    for sent_type in sentiment_types:
        print("\t{}: \tTP: {};\tFP: {};\tFN: {};\tprecision: {:.2f};\trecall: {:.2f};\tf1: {:.2f};\t{}".format(
            sent_type,
            scores[sent_type]["tp"],
            scores[sent_type]["fp"],
            scores[sent_type]["fn"],
            scores[sent_type]["p"],
            scores[sent_type]["r"],
            scores[sent_type]["f1"],
            scores[sent_type]["tp"] +
            scores[sent_type][
                "fp"]))

def evaluate_extraction(samples, predictions_b):
    scores = {"tp": 0, "fp": 0, "fn": 0}
    for label, pred in zip (samples, predictions_b):
        pred_terms = {term_pred[0] for term_pred in pred["targets"]}
        gt_terms = {term_gt[1] for term_gt in label["targets"]}

        scores["tp"] += len(pred_terms & gt_terms)
        scores["fp"] += len(pred_terms - gt_terms)
        scores["fn"] += len(gt_terms - pred_terms)

    precision = 100 * scores["tp"] / (scores["tp"] + scores["fp"])
    recall = 100 * scores["tp"] / (scores["tp"] + scores["fn"])
    f1 = 2 * precision * recall / (precision + recall)

    print(f"Aspect Extraction Evaluation")

    print(
        "\tAspects\t TP: {};\tFP: {};\tFN: {}".format(
            scores["tp"],
            scores["fp"],
            scores["fn"]))
    print(
        "\t\tprecision: {:.2f};\trecall: {:.2f};\tf1: {:.2f}".format(
            precision,
            recall,
            f1))

# TASK C-D (DistillBERT fine tuning)

## Dataset

In [ ]:
class TaskCDDataset (Dataset):
    def __init__(self,path, tokenizer, max_len,model_b):
        self.tokenizer = tokenizer
        self.model_b = model_b

        self.texts, self.tags = self.load_data(path)
        self.mlb = MultiLabelBinarizer()
        self.encode_tags()
        self.max_len = max_len

    def encode_tags(self):
        self.tags = self.mlb.fit_transform(self.tags)

    def decode_tag(self,tag):
        return self.mlb.inverse_transform(tag.reshape(1,-1))

    def load_data(self,datapath):
        sentences = []     
        texts = []
        tags = []
        with open(datapath) as json_file:
            list_of_sentences = json.load(json_file)
            for obj in list_of_sentences:
                texts.append(obj['text'])
                if self.model_b:
                    tags.append([cat[0]+"-"+cat[1] for cat in obj['categories']])
                else:
                    tags.append([cat[0] for cat in obj['categories']])

                
        return texts, tags  
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item_idx):
        text = self.texts[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, 
            truncation=True,
            return_tensors = 'pt'
          )
        
        input_ids = inputs['input_ids'].flatten()
        attn_mask = inputs['attention_mask'].flatten()
        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': torch.tensor(self.tags[item_idx], dtype=torch.float)
        }
        

In [ ]:
class TaskCDDataModule (pl.LightningDataModule):
    
    def __init__(self,training_file, dev_file, tokenizer,batch_size=16,max_token_len=200):
        super().__init__()
        self.training_file = training_file
        self.dev_file = dev_file
        
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self):
        self.train_dataset = TaskCDDataset(self.training_file, tokenizer=self.tokenizer,max_len = self.max_token_len,model_b=model_b)
        self.val_dataset  = TaskCDDataset(self.dev_file, tokenizer=self.tokenizer,max_len = self.max_token_len,model_b=model_b)
           
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size = self.batch_size,shuffle = True , num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size= 16)

## Model

Set up the classifier


In [ ]:
class TaskCDClassifier(pl.LightningModule):
    def __init__(self, n_classes=5, steps_per_epoch=None, n_epochs=3, lr=2e-5,device="cuda" ):
        super().__init__()
        self.bert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased',num_labels=n_classes)
        #self.classifier = nn.Linear(self.bert.config.hidden_size,n_classes) # outputs = number of labels
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.lr = lr
        self.criterion = nn.BCEWithLogitsLoss()
        self.to(device)
    
    def forward(self,input_ids, attn_mask):
        output = self.bert(input_ids = input_ids ,attention_mask = attn_mask)
        #pooler output --> Last layer hidden-state of the first token of the sequence (classification token) further processed 
        #by a Linear layer and a Tanh activation function. The Linear layer weights are trained from the next 
        #sentence prediction (classification) objective during pretraining.
        #output = self.classifier(output.pooler_output)
        output = output['logits']        
        return output
    
    
    def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('train_loss',loss , prog_bar=True,logger=True)
        
        return {"loss" :loss, "predictions":outputs, "labels": labels }


    def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('val_loss',loss , prog_bar=True,logger=True)
        
        return loss
    
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters() , lr=self.lr)
        return optimizer
    

Initialize the parameters that will be use for training


In [ ]:
bert_tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
N_EPOCHS = 3
BATCH_SIZE = 32
MAX_LEN = 100
LR = 2e-05

Instantiate and set up the data_module


In [ ]:
TaskCD_data_module = TaskCDDataModule(
    dataset_folder+"/restaurants_train.json",
    dataset_folder+"/restaurants_dev.json",
    bert_tokenizer,
    BATCH_SIZE,
    MAX_LEN
    )
TaskCD_data_module.setup()

Instantiate the classifier model


In [ ]:
train_d = TaskCD_data_module.train_dataset
steps_per_epoch = len(train_d.texts)//BATCH_SIZE
from transformers import DistilBertForSequenceClassification
model = TaskCDClassifier(n_classes=train_d.tags.shape[1], steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

## Training

Instantiate the Model Trainer


In [ ]:
trainer = pl.Trainer(max_epochs = N_EPOCHS , gpus = 1,progress_bar_refresh_rate = 10)
model.parameters


Train the Classifier Model


In [ ]:
trainer.fit(model, TaskCD_data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                                | Params
------------------------------------------------------------------
0 | bert      | DistilBertForSequenceClassification | 65.8 M
1 | criterion | BCEWithLogitsLoss                   | 0     
------------------------------------------------------------------
65.8 M    Trainable params
0         Non-trainable params
65.8 M    Total params
263.188   Total estimated model params size (MB)


## Evaluation

In [ ]:
pred_outs, true_labels = [], []
model.cuda()
for batch in TaskCD_data_module.val_dataloader():
    # Unpack the inputs from our dataloader
    b_input_ids = batch["input_ids"].cuda()
    b_attn_mask = batch["attention_mask"].cuda()
    b_labels = batch["label"].cuda()
    
    #print(b_input_ids)
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        pred_out = model(b_input_ids,b_attn_mask)
        pred_out = torch.sigmoid(pred_out)
        # Move predicted output and labels to CPU
        
        pred_out = pred_out.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

    pred_outs.append(pred_out)
    true_labels.append(label_ids)

flat_pred_outs = np.concatenate(pred_outs, axis=0)

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

convert probabilities into 0 or 1 based on a threshold value

In [ ]:
threshold  = np.arange(0.2,0.51,0.01)

# convert probabilities into 0 or 1 based on a threshold value
def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

scores=[] # Store the list of f1 scores for prediction on each threshold

#convert labels to 1D array
y_true = flat_true_labels.ravel() 

for thresh in threshold:
    
    #classes for each threshold
    pred_bin_label = classify(flat_pred_outs,thresh) 

    #convert to 1D array
    y_pred = np.array(pred_bin_label).ravel()

    scores.append(f1_score(y_true,y_pred))

# find the optimal threshold
opt_thresh = threshold[scores.index(max(scores))]
print(f'Optimal Threshold Value = {opt_thresh}')

Optimal Threshold Value = 0.23000000000000004


In [ ]:
#predictions for optimal threshold
y_pred_labels = classify(flat_pred_outs,opt_thresh)

print(y_pred_labels)
y_pred = np.array(y_pred_labels).ravel() # Flatten

metr = {}
metr['f1'] = f1_score(y_true,y_pred, average="macro")

print(metr['f1'])

[[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 

In [ ]:
import pandas as pd
y_pred = TaskCD_data_module.val_dataset.mlb.inverse_transform(np.array(y_pred_labels))
y_act = TaskCD_data_module.val_dataset.mlb.inverse_transform(flat_true_labels)


df = pd.DataFrame({'Body':TaskCD_data_module.val_dataset.texts,'Actual Tags':y_act,'Predicted Tags':y_pred})

In [ ]:
pd.set_option('display.max_rows', 500)
df.sample(10)

,Body,Actual Tags,Predicted Tags
20,"Good food at the restaurant (a bit expensive, ...","(food-positive, price-negative)","(food-positive,)"
447,But don't ever order bacon late at nite (eithe...,"(food-negative,)","(food-negative,)"
456,I'm still mad that i had to pay for lousy food.,"(food-negative,)","(food-negative,)"
528,Restaurant was dirty and unkempt.,"(ambience-negative,)","(service-negative,)"
335,There are many Thai places in the city but so ...,"(anecdotes/miscellaneous-positive,)","(anecdotes/miscellaneous-neutral,)"
224,"Been there, done that, and New York, it's not ...","(anecdotes/miscellaneous-neutral,)","(anecdotes/miscellaneous-neutral,)"
21,"In fact, two people could really share one plate.","(food-positive,)",()
444,"The food is very good for it's price, better t...","(food-positive, price-positive)","(food-positive,)"
154,We visit at least once a month.,"(anecdotes/miscellaneous-neutral,)","(anecdotes/miscellaneous-neutral, anecdotes/mi..."
268,Others have to go to other restaurants and fee...,"(anecdotes/miscellaneous-neutral,)",()


In [ ]:
TaskCD_data_module.val_dataset.mlb.classes_

array(['ambience-conflict', 'ambience-negative', 'ambience-neutral',
       'ambience-positive', 'anecdotes/miscellaneous-conflict',
       'anecdotes/miscellaneous-negative',
       'anecdotes/miscellaneous-neutral',
       'anecdotes/miscellaneous-positive', 'food-conflict',
       'food-negative', 'food-neutral', 'food-positive', 'price-conflict',
       'price-negative', 'price-neutral', 'price-positive',
       'service-conflict', 'service-negative', 'service-neutral',
       'service-positive'], dtype=object)

In [ ]:
torch.save(model.state_dict(), root_folder+'/model/TASKCD_model_b={}_f1_{:0.4f}.pt'.format(str(model_b), metr["f1"])) # save the model state

## docker

In [ ]:
def load_data(data_path):
    with open(data_path) as json_file:
        list_of_sentences = json.load(json_file)
    return list_of_sentences

In [ ]:
class PreprocessCD():
    def __init__(self, sentences):
        self.texts, self.tags = self.load_data(sentences)
        self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
        self.max_len = 200
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit_transform([['ambience-conflict', 'ambience-negative', 'ambience-neutral',
       'ambience-positive', 'anecdotes/miscellaneous-conflict',
       'anecdotes/miscellaneous-negative',
       'anecdotes/miscellaneous-neutral',
       'anecdotes/miscellaneous-positive', 'food-conflict',
       'food-negative', 'food-neutral', 'food-positive', 'price-conflict',
       'price-negative', 'price-neutral', 'price-positive',
       'service-conflict', 'service-negative', 'service-neutral',
       'service-positive']])
        self.encodings = self.encode_texts()
        
    def encode_texts(self):
        encodings = []
        for item_idx,text in enumerate(self.texts):
            inputs = self.tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True, # Add [CLS] [SEP]
                max_length= self.max_len,
                padding = 'max_length',
                return_token_type_ids= False,
                return_attention_mask= True, # Differentiates padded vs normal token
                truncation=True, # Truncate data beyond max length
                return_tensors = 'pt' # PyTorch Tensor format
            )
            
            input_ids = inputs['input_ids'].flatten()
            attn_mask = inputs['attention_mask'].flatten()
            
            encodings.append({
                'input_ids': input_ids ,
                'attention_mask': attn_mask,
            })
        return encodings
        

    def load_data(self,list_of_sentences):
        texts = []
        tags = []

        for obj in list_of_sentences:
            texts.append(obj['text'])            
        return texts, tags  
    

    def __len__(self):
        return len(self.labels)


def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

def predictCD(samples: List[Dict]) -> List[Dict]:
    targets = []
    prep = PreprocessCD(samples)
    model.eval()
    model.cpu()
    for i,batch in enumerate(prep.encodings):

        json_pred = {"targets":[], "categories": []}
        b_input_ids = batch["input_ids"]
        b_attn_mask = batch["attention_mask"]
        
        ids = torch.unsqueeze(torch.tensor(b_input_ids),0).cpu()
        attmasks = torch.unsqueeze(torch.tensor(b_attn_mask),0).cpu()
        # Forward pass, calculate logit predictions
        pred_out = model(ids,attmasks)
        pred_out = torch.sigmoid(pred_out)
        
        pred_out = pred_out.detach().cpu().numpy()

        y_pred_labels = classify(pred_out,0.23)
        
        y_pred = prep.mlb.inverse_transform(np.array(y_pred_labels))
        for pred in y_pred:
            try:
                asd = pred[0].split("-")
                json_pred["categories"].append((asd[0],asd[1]))
            except:
                continue

        targets.append(json_pred)
    
    return targets

            
a = load_data(dataset_folder+"/restaurants_dev.json")
random.shuffle(a)
t = predictCD(a)

for obj,o in zip(a[:30],t):
    print(obj["categories"],"SEP",o["categories"])
# print(t['categories'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[['service', 'conflict']] SEP [('service', 'negative')]
[['anecdotes/miscellaneous', 'positive']] SEP [('anecdotes/miscellaneous', 'positive')]
[['food', 'conflict']] SEP [('food', 'positive')]
[['food', 'positive'], ['service', 'positive']] SEP []
[['food', 'positive'], ['ambience', 'positive']] SEP [('food', 'positive')]
[['anecdotes/miscellaneous', 'neutral']] SEP []
[['service', 'conflict']] SEP [('service', 'negative')]
[['anecdotes/miscellaneous', 'positive']] SEP [('anecdotes/miscellaneous', 'positive')]
[['anecdotes/miscellaneous', 'positive']] SEP [('anecdotes/miscellaneous', 'neutral')]
[['service', 'negative'], ['anecdotes/miscellaneous', 'negative']] SEP [('food', 'negative')]
[['service', 'negative']] SEP []
[['anecdotes/miscellaneous', 'positive']] SEP [('food', 'positive')]
[['anecdotes/miscellaneous', 'positive']] SEP []
[['food', 'positive'], ['service', 'negative']] SEP [('food', 'negative')]
[['food', 'positive'], ['ambience', 'positive']] SEP [('food', 'positive')]


In [ ]:
evaluate_sentiment(a,t,mode="Category Extraction")

Category Extraction Evaluation

	ALL	 TP: 313;	FP: 54;	FN: 231
		(m avg): precision: 85.29;	recall: 57.54;	f1: 68.72 (micro)
		(M avg): precision: 60.97;	recall: 40.25;	f1: 46.18 (Macro)

	anecdotes/miscellaneous: 	TP: 118;	FP: 13;	FN: 73;	precision: 90.08;	recall: 61.78;	f1: 73.29;	131
	price: 	TP: 0;	FP: 0;	FN: 53;	precision: 0.00;	recall: 0.00;	f1: 0.00;	0
	food: 	TP: 181;	FP: 35;	FN: 43;	precision: 83.80;	recall: 80.80;	f1: 82.27;	216
	ambience: 	TP: 14;	FP: 6;	FN: 62;	precision: 70.00;	recall: 18.42;	f1: 29.17;	20


In [ ]:
def evaluate_sentiment(samples, predictions_b, mode="Aspect Sentiment"):
    scores = {}
    if mode == 'Category Extraction':
        sentiment_types = ["anecdotes/miscellaneous", "price", "food", "ambience"]
    else:
        sentiment_types = ["positive", "negative", "neutral", "conflict"]
    scores = {sent: {"tp": 0, "fp": 0, "fn": 0} for sent in sentiment_types + ["ALL"]}
    for label, pred in zip(samples, predictions_b):
        for sentiment in sentiment_types:
            if mode == "Aspect Sentiment":
                pred_sent = {(term_pred[0], term_pred[1]) for term_pred in pred["targets"] if
                                    term_pred[1] == sentiment}
                gt_sent = {(term_pred[1], term_pred[2]) for term_pred in label["targets"] if
                                    term_pred[2] == sentiment}
            elif mode == 'Category Extraction' and "categories" in label:
                pred_sent = {(term_pred[0]) for term_pred in pred["categories"] if
                                term_pred[0] == sentiment}
                gt_sent = {(term_pred[0]) for term_pred in label["categories"] if
                                term_pred[0] == sentiment}
            elif "categories" in label:
                pred_sent = {(term_pred[0], term_pred[1]) for term_pred in pred["categories"] if
                                term_pred[1] == sentiment}
                gt_sent = {(term_pred[0], term_pred[1]) for term_pred in label["categories"] if
                                term_pred[1] == sentiment}
            else:
                continue

            scores[sentiment]["tp"] += len(pred_sent & gt_sent)
            scores[sentiment]["fp"] += len(pred_sent - gt_sent)
            scores[sentiment]["fn"] += len(gt_sent - pred_sent)

    # Compute per sentiment Precision / Recall / F1
    for sent_type in scores.keys():
        if scores[sent_type]["tp"]:
            scores[sent_type]["p"] = 100 * scores[sent_type]["tp"] / (scores[sent_type]["fp"] + scores[sent_type]["tp"])
            scores[sent_type]["r"] = 100 * scores[sent_type]["tp"] / (scores[sent_type]["fn"] + scores[sent_type]["tp"])
        else:
            scores[sent_type]["p"], scores[sent_type]["r"] = 0, 0

        if not scores[sent_type]["p"] + scores[sent_type]["r"] == 0:
            scores[sent_type]["f1"] = 2 * scores[sent_type]["p"] * scores[sent_type]["r"] / (
                    scores[sent_type]["p"] + scores[sent_type]["r"])
        else:
            scores[sent_type]["f1"] = 0

    # Compute micro F1 Scores
    tp = sum([scores[sent_type]["tp"] for sent_type in sentiment_types])
    fp = sum([scores[sent_type]["fp"] for sent_type in sentiment_types])
    fn = sum([scores[sent_type]["fn"] for sent_type in sentiment_types])

    if tp:
        precision = 100 * tp / (tp + fp)
        recall = 100 * tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)

    else:
        precision, recall, f1 = 0, 0, 0

    scores["ALL"]["p"] = precision
    scores["ALL"]["r"] = recall
    scores["ALL"]["f1"] = f1
    scores["ALL"]["tp"] = tp
    scores["ALL"]["fp"] = fp
    scores["ALL"]["fn"] = fn

    # Compute Macro F1 Scores
    scores["ALL"]["Macro_f1"] = sum([scores[ent_type]["f1"] for ent_type in sentiment_types])/len(sentiment_types)
    scores["ALL"]["Macro_p"] = sum([scores[ent_type]["p"] for ent_type in sentiment_types])/len(sentiment_types)
    scores["ALL"]["Macro_r"] = sum([scores[ent_type]["r"] for ent_type in sentiment_types])/len(sentiment_types)

    print(f"{mode} Evaluation\n")

    print(
        "\tALL\t TP: {};\tFP: {};\tFN: {}".format(
            scores["ALL"]["tp"],
            scores["ALL"]["fp"],
            scores["ALL"]["fn"]))
    print(
        "\t\t(m avg): precision: {:.2f};\trecall: {:.2f};\tf1: {:.2f} (micro)".format(
            precision,
            recall,
            f1))
    print(
        "\t\t(M avg): precision: {:.2f};\trecall: {:.2f};\tf1: {:.2f} (Macro)\n".format(
            scores["ALL"]["Macro_p"],
            scores["ALL"]["Macro_r"],
            scores["ALL"]["Macro_f1"]))

    for sent_type in sentiment_types:
        print("\t{}: \tTP: {};\tFP: {};\tFN: {};\tprecision: {:.2f};\trecall: {:.2f};\tf1: {:.2f};\t{}".format(
            sent_type,
            scores[sent_type]["tp"],
            scores[sent_type]["fp"],
            scores[sent_type]["fn"],
            scores[sent_type]["p"],
            scores[sent_type]["r"],
            scores[sent_type]["f1"],
            scores[sent_type]["tp"] +
            scores[sent_type][
                "fp"]))